<a href="https://colab.research.google.com/github/hieunguyen7337/LLM_RL/blob/main/Hangman_game_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import gc
import random
import json
import torch

In [2]:
!git clone https://github.com/hieunguyen7337/LLM_RL.git

Cloning into 'LLM_RL'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 32 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (32/32), 201.84 KiB | 830.00 KiB/s, done.
Resolving deltas: 100% (7/7), done.


In [3]:
from huggingface_hub import notebook_login

notebook_login()

#Load model

In [4]:
# model_name = "Qwen/Qwen3-0.6B"
# model_name = "Qwen/Qwen3-8B"
model_name = "Qwen/Qwen2.5-0.5B"
# model_name = "google/gemma-3-270m"

In [ ]:
# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
model.eval()

#Preprocess Data

In [5]:
with open('/content/LLM_RL/random_words_list.txt', 'r') as f:
  word_list = f.readlines()

# Remove newline characters
word_list = [word.strip() for word in word_list]

In [7]:
print(f"\nNumber of words: {len(word_list)}")


Number of words: 2466


In [8]:
# Check for duplicate words
if len(word_list) == len(set(word_list)):
  print("No duplicate words found in the list.")
else:
  print("Duplicate words found in the list.")

No duplicate words found in the list.


In [9]:
# Remove words with hyphens
word_list = [word for word in word_list if '-' not in word and ' ' not in word]

# Check the number of words remaining
print(f"\nNumber of words remaining after removing hyphens: {len(word_list)}")


Number of words remaining after removing hyphens: 2444


In [10]:
# # Get a specified number of random words (e.g., 5)
# # num_random_words = 200
# num_random_words = 2
# random_words = random.sample(word_list, num_random_words)
# # print(f"\n{num_random_words} random words from the list")

In [37]:
random_words = word_list

In [45]:
# Create a list to store the input texts
input_texts = []
input_words = []

for secret_word in random_words:
  for i in range(6):
    word_length = len(secret_word)
    current_state = ["_" for _ in range(word_length)]
    incorrect_guesses_remaining = 6
    guessed_letters = []

    # Simulate some random guesses to create a game in progress
    num_simulated_guesses = random.randint(0, len(secret_word)) # Simulate 0 to 3 guesses
    # num_simulated_guesses = 20
    available_letters = list("abcdefghijklmnopqrstuvwxyz")
    correct_letters = list(set(secret_word))

    for _ in range(num_simulated_guesses):
      if random.random() > 0.5:
        guess = random.choice(available_letters)
      else:
        guess = random.choice(correct_letters)

      correct_letters.remove(guess) if guess in correct_letters else None
      available_letters.remove(guess) if guess in available_letters else None

      guessed_letters.append(guess.upper())

      if guess in secret_word:
        for i in range(word_length):
          if secret_word[i] == guess:
            current_state[i] = guess

      else:
        incorrect_guesses_remaining -= 1
        if incorrect_guesses_remaining == 0:
          break # Game over during simulation

      if len(correct_letters) == 0:
        break

    # Create the input text for the prompt
    input_text = f"""You are playing a game of Hangman.

Your task is to guess a single character.

The word has a certain number of letters.
The current state of the word is shown with guessed letters filled in and blanks for the unknown letters.
The number of incorrect guesses remaining is listed.
All letters that have been guessed so far are listed.

You will format your response as a single uppercase letter at the end

The word has {word_length} letters.
The current state is: {' '.join(current_state)}
Incorrect guesses remaining: {incorrect_guesses_remaining}
Guessed letters: {guessed_letters}

Correct response:"""
    # if True:
    if incorrect_guesses_remaining > 0 and len(correct_letters) > 0 and input_text not in input_texts:
      input_texts.append(input_text)
      input_words.append(secret_word) # Store both prompt and word

In [53]:
# Print the first generated input text to verify
print(len(input_texts))
print()
i = 2
print(input_texts[i])
print(input_words[i])

10519

You are playing a game of Hangman.

Your task is to guess a single character.

The word has a certain number of letters.
The current state of the word is shown with guessed letters filled in and blanks for the unknown letters.
The number of incorrect guesses remaining is listed.
All letters that have been guessed so far are listed.

You will format your response as a single uppercase letter at the end

The word has 8 letters.
The current state is: _ _ _ _ _ _ _ _
Incorrect guesses remaining: 6
Guessed letters: []

Correct response:
imported


In [60]:
# Save both lists in one file
with open("hangman_data.json", "w", encoding="utf-8") as f:
    json.dump({"input_texts": input_texts, "input_words": input_words}, f, indent=4, ensure_ascii=False)

#Load Preprocessed Data

In [6]:
# Load the JSON file
with open("/content/LLM_RL/hangman_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [7]:
input_texts = data["input_texts"]
input_words = data["input_words"]

In [8]:
i = 3
print(input_texts[i])
print(input_words[i])

You are playing a game of Hangman.

Your task is to guess a single character.

The word has a certain number of letters.
The current state of the word is shown with guessed letters filled in and blanks for the unknown letters.
The number of incorrect guesses remaining is listed.
All letters that have been guessed so far are listed.

You will format your response as a single uppercase letter at the end

The word has 8 letters.
The current state is: _ _ p _ _ t _ _
Incorrect guesses remaining: 6
Guessed letters: ['T', 'P']

Correct response:
imported


#Inference Model

In [13]:
# for text in input_texts:
#   print(tokenizer(text, return_tensors="pt")["input_ids"].size()[1])
#   print()

In [9]:
tokenizer.padding_side = "left"

In [10]:
# Divide input_texts into 10 parts
num_parts = 10
part_size = len(input_texts) // num_parts
input_parts = [input_texts[i*part_size:(i+1)*part_size] for i in range(num_parts)]

# Tokenize each part and store in a list
input_ids_list = [tokenizer(part, return_tensors="pt", padding=True).to(model.device) for part in input_parts]

In [11]:
input_ids_list[0]

{'input_ids': tensor([[151643, 151643, 151643,  ...,  33092,   2033,     25],
        [151643, 151643, 151643,  ...,  33092,   2033,     25],
        [151643, 151643, 151643,  ...,  33092,   2033,     25],
        ...,
        [151643, 151643, 151643,  ...,  33092,   2033,     25],
        [151643, 151643, 151643,  ...,  33092,   2033,     25],
        [151643, 151643, 151643,  ...,  33092,   2033,     25]],
       device='cuda:0'), 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]], device='cuda:0')}

In [12]:
for i in range(num_parts):
  print(input_ids_list[i]["input_ids"].size())

torch.Size([1051, 157])
torch.Size([1051, 153])
torch.Size([1051, 153])
torch.Size([1051, 155])
torch.Size([1051, 154])
torch.Size([1051, 153])
torch.Size([1051, 158])
torch.Size([1051, 153])
torch.Size([1051, 158])
torch.Size([1051, 161])


In [13]:
outputs = []
for i in range(5):
  print("part",i)
  input_ids = input_ids_list[i]
  outputs.append(model.generate(
              **input_ids,
              max_new_tokens=1))
  gc.collect()
  torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


part 0


In [14]:
outputs[0]

tensor([[151643, 151643, 151643,  ...,   1599,    271,   2610],
        [151643, 151643, 151643,  ...,    716,    716,    716],
        [151643, 151643, 151643,  ...,    472, 151643, 151643],
        ...,
        [151643, 151643, 151643,  ...,    479, 151643, 151643],
        [151643, 151643, 151643,  ...,    435,    271,    785],
        [151643, 151643, 151643,  ...,    431, 151643, 151643]],
       device='cuda:0')

In [15]:
decoded_texts = [tokenizer.batch_decode(output, skip_special_tokens=True) for output in outputs]

In [16]:
decoded_texts = [item for sublist in decoded_texts for item in sublist] # flatten the list

In [17]:
len(decoded_texts)

1051

In [18]:
letter_preds = [text.split(":")[-1].strip() for text in decoded_texts]

In [19]:
is_single_letter = [len(pred) == 1 and pred.isalpha() for pred in letter_preds]
# print(is_single_letter)
print(letter_preds)
print(f"Number of single letters: {sum(is_single_letter)}")

['X\n\nYou', '_ _ _', 'H', 'p\n\nYou', 'i', 'A', 'A', 'V', 'pLty', 't\n\nThe', '_ a l', 'H\n\nYou', 'J', 'A\n\nYou', 'A', 'A\n\nYou', '_ _ _', 'Q\n\nYou', 'y\n\nYou', '_ u n', 'U\n\nYou', 'F\n\nYou', 'C', 'H', '_ i _', '_ i _', 'f\n\nYou', 'A', 't\n\nThe', '_ _ r', 't\n\nTo', 'r\n\nYou', 't h e', 'H\n\nYou', 'A', 's\n\nYou', 'a\n\nYou', 'D\n\nYou', 'J\n\nYou', '_ _ _', 'h\n\nYou', '_ l c', 'W\n\nThe', 'H\n\nYou', 'G', 'I\n\nYou', 'n', 'J\n\nYou', 'B', 'n', 'J\n\nYou', 'A', '_ _ a', 'A', '_ r _', 'm\n\nYou', '_ _ _', '_ _ _', 'o\n\nThe', 'c', 'H', 'A', 'B\n\nYou', '_ l i', 'Z\n\nYou', 'I\n\nYou', 'c', 'c\n\nThe', 'E', 'c', 'U', '_ _ _', 'A\n\nYou', 'M', 'W\n\nYou', 'D', 'r', 'YB', '_ _ _', 'p\n\nThe', 'rly', 'F\n\nYou', 'S\n\nYou', 'R\n\nYou', 'R\n\nYou', 'n\n\nYou', 'T\n\nYou', 'L', 'Q', 'U', 'A\n\nYou', '_ _ _', 'S\n\nYou', 'A\n\nYou', 'A', 'I\n\nYou', 'J\n\nYou', '_ i n', 'P\n\nYou', 'Q\n\nYou', 'F\n\nYou', '_ _ l', 'Z', 'H\n\nYou', '_ _ l', 'r\n\nYou', 'L', 't', 't', 'p', 'V\n\nYou'

In [5]:
output_3 = ['X\n\nYou', '_ _ _', 'H', 'p\n\nYou', 'i', 'A', 'A', 'V', 'pLty', 't\n\nThe', '_ a l', 'H\n\nYou', 'J', 'A\n\nYou', 'A', 'A\n\nYou', '_ _ _', 'Q\n\nYou', 'y\n\nYou', '_ u n', 'U\n\nYou', 'F\n\nYou', 'C', 'H', '_ i _', '_ i _', 'f\n\nYou', 'A', 't\n\nThe', '_ _ r', 't\n\nTo', 'r\n\nYou', 't h e', 'H\n\nYou', 'A', 's\n\nYou', 'a\n\nYou', 'D\n\nYou', 'J\n\nYou', '_ _ _', 'h\n\nYou', '_ l c', 'W\n\nThe', 'H\n\nYou', 'G', 'I\n\nYou', 'n', 'J\n\nYou', 'B', 'n', 'J\n\nYou', 'A', '_ _ a', 'A', '_ r _', 'm\n\nYou', '_ _ _', '_ _ _', 'o\n\nThe', 'c', 'H', 'A', 'B\n\nYou', '_ l i', 'Z\n\nYou', 'I\n\nYou', 'c', 'c\n\nThe', 'E', 'c', 'U', '_ _ _', 'A\n\nYou', 'M', 'W\n\nYou', 'D', 'r', 'YB', '_ _ _', 'p\n\nThe', 'rly', 'F\n\nYou', 'S\n\nYou', 'R\n\nYou', 'R\n\nYou', 'n\n\nYou', 'T\n\nYou', 'L', 'Q', 'U', 'A\n\nYou', '_ _ _', 'S\n\nYou', 'A\n\nYou', 'A', 'I\n\nYou', 'J\n\nYou', '_ i n', 'P\n\nYou', 'Q\n\nYou', 'F\n\nYou', '_ _ l', 'Z', 'H\n\nYou', '_ _ l', 'r\n\nYou', 'L', 't', 't', 'p', 'V\n\nYou', 'p', 'p\n\nYou', 'T\n\nYou', 'W\n\nThe', '_ _ e', 'Q\n\nYou', 'WNR', 'B\n\nYou', 'H', 'l\n\nYou', 'Z\n\nYou', 'H', 'h\n\nTo', 'A', '_ e r', '_ e r', '_ e r', '_ e r', 'P', '_ _ _', '_ r _', 'q\n\nTo', 't\n\nYou', 'U', 'A\n\nYou', '_ _ r', 'V\n\nYou', 'R\n\nYou', 'Z', '_ _ _', 'G', 'R\n\nYou', 'I\n\nYou', '_ e _', '_ _ _', '_ e t', '_ _ _', '_ _ _', '_ _ _', 'B', 'QN', 'V\n\nYou', '_ _ _', 'S\n\nYou', 'K\n\nYou', 'K\n\nYou', 'F\n\nYou', 'K\n\nYou', 'E\n\nYou', 'F\n\nYou', 't\n\nThe', 'A', 't', 't', 'P', 'F\n\nYou', 'A\n\nYou', 'a\n\nYou', 'a\n\nYou', 'X\n\nYou', '_ f f', 'K', '_ f f', 'A', '_ _ _', 'J\n\nYou', 't\n\nThe', 'X\n\nThe', 't\n\nThe', 'm', 'N\n\nYou', '_ _ _', '_ n s', 'N\n\nYou', 'z\n\nYou', '_ _ _', 'H\n\nYou', 'RAC', 'A\n\nYou', 'A', 'A', '_ e s', 'W', 't o o', 't _ _', 't _ _', 't', '_ _ _', 'H\n\nYou', 'h', 'A', 'H', 'C\n\nYou', 'A', 'U', 'A', 'FV', 'U\n\nYou', 'I\n\nYou', 'A', 'M\n\nYou', 'T\n\nYou', 'M\n\nYou', '_ _ _', '_ _ _', 'E', 'U', 'U', 'U\n\nYou', 'H', 'U', 't\n\nYou', 'b\n\nYou', 'R', 'C', 'R', 'E', '_ r i', 'R\n\nYou', 'C\n\nYou', 'T\n\nYou', 'c\n\nYou', 'E\n\nYou', '`U`', 'B\n\nYou', 'H', '_ _ _', 'B\n\nYou', 'E\n\nYou', 'A', 'E\n\nYou', 'X\n\nYou', 'E\n\nYou', 'U', 'A', '_ _ _', '_ _ _', 'V', '_ _ _', '_ _ e', 'W\n\nYou', 'E', 'H\n\nYou', 't', '_ _ _', 'Z\n\nYou', 'U', 's\n\nYou', 'A', 'G\n\nYou', 't\n\nThe', 't\n\nThe', '_ a _', 'H', 'tng', 'D', 'U', '_ _ r', 'N\n\nYou', 'G', '_ _ _', 'A', 'A\n\nYou', '_ p p', '_ p p', 'JSS', 'H', 'A', 'A', 'T\n\nYou', 'A\n\nYou', '_ u _', 'm\n\nYou', 'N', 'J\n\nYou', '_ _ _', 'A\n\nYou', '_ u t', 't\n\nThe', '_ _ _', 'R', '_ _ _', '_ r _', 'W\n\nYou', 'G', 'J', 'A', 'F\n\nYou', 'E\n\nYou', 'd\n\nYou', '_ _ _', 'Z', 'J\n\nYou', '_ _ b', 'U', 'd', '_ _ _', 'A', '_ _ v', '_ _ _', 'L\n\nYou', 'F\n\nYou', 'O\n\nYou', 'C', 'A', 'F\n\nYou', 'D', 'K\n\nYou', 'N', 'Z\n\nThe', 'r', '_ _ _', 'r\n\nThe', '_ _ _', 'T\n\nYou', 'X\n\nYou', 't\n\nThe', 'X\n\nYou', '_ _ e', 't\n\nThe', '_ _ _', 'E\n\nYou', '_ _ _', 'p', 'p\n\nYou', 'W', 'W', 'H\n\nYou', 'H\n\nYou', 'R\n\nYou', 'H', 'V\n\nYou', 'R', 'A\n\nYou', 'J', '_ _ _', 'M\n\nYou', 'm\n\nThe', '_ _ _', 'E', '_ _ _', 'H\n\nYou', 'VJ', 'F\n\nThe', 'F\n\nYou', 'F\n\nYou', 'r\n\nYou', 'F\n\nYou', 'U', 'R', 'H', 'G', 'E\n\nYou', 'O\n\nYou', 'i\n\nYou', 'G', 'B', 'X\n\nYou', 'A\n\nYou', 'N\n\nYou', 'K\n\nYou', '_ _ _', 'Y\n\nYou', 'A\n\nYou', 't\n\nYou', 'c\n\nYou', 'K\n\nYou', 'T\n\nYou', '_ _ i', 'P', '_ o i', 'X\n\nYou', '_ _ i', 'r\n\nYou', 't o', 'H\n\nYou', 'ZD', 'X\n\nYou', 'E\n\nYou', 'd\n\nYou', 'O', 'X\n\nYou', 'i\n\nYou', 'L', 'S\n\nYou', 'W\n\nThe', '_ _ g', 'U', 'O\n\nYou', 'V', 'T\n\nYou', 'R', '_ _ _', 'I\n\nYou', '_ _ k', 'K\n\nYou', 'T\n\nYou', 'T', 'QAL', 'B\n\nYou', 'L\n\nYou', 'Q\n\nYou', 'YC', 'RLEBL', 'O\n\nYou', 'Z\n\nYou', 'W', 'l _ _', 'W', '_ _ v', '_ _ v', 'V', 'D', 'D', 'W\n\nYou', 'd\n\nYou', '_ _ l', '_ _ l', 'J\n\nYou', 'O\n\nYou', '_ _ _', 'P', 'H', 'M\n\nYou', '_ i _', 'c\n\nYou', 'c\n\nYou', 'X\n\nYou', 'M\n\nYou', '_ _ c', 'I\n\nYou', 'I\n\nYou', 'X\n\nThe', 'U', 'h', '_ _ _', 'C\n\nYou', 'i\n\nYou', '_ _ _', 'r\n\nThe', '_ r _', 'V\n\nYou', '`G`', '`G`', '_ _ i', '_ _ _', 'FRR', 'F\n\nYou', '_ p r', '_ _ _', 'N\n\nYou', 'O', 'O\n\nYou', 'H\n\nYou', '_ o i', 'c\n\nYou', 'n\n\nYou', '_ _ l', 'l\n\nYou', 'H', 'F\n\nYou', '_ e p', 'F\n\nYou', 'L\n\nYou', 'Y\n\nYou', 'Q\n\nYou', 'A', 'M\n\nYou', 's\n\nYou', '_ _ _', 'd', 'o\n\nThe', 'H\n\nThe', 'F', 'O', 'l\n\nYou', '_ _ _', 'QLT', '_ _ _', 'h\n\nYou', '_ i g', 'h\n\nTo', 't\n\nThe', '_ _ _', 'y\n\nYou', 'T\n\nYou', '_ _ _', 'A\n\nYou', 'ODD', 'T\n\nYou', '_ a s', 'A', 'L', 't\n\nThe', 'h\n\nTo', 'W', 'H', 'OQ', 'r\n\nYou', 'H\n\nYou', 'H\n\nYou', 'H\n\nYou', 'D\n\nYou', 'H\n\nYou', 'J\n\nYou', 'J\n\nYou', 'W\n\nYou', 'N', 'A', '_ l _', 'X\n\nYou', 'e\n\nThe', 'E', 'M', '_ _ _', 'W\n\nThe', 'U\n\nYou', 'A', 'P', 'M\n\nYou', 'm', 'r\n\nYou', 'H', 'n\n\nYou', 'r\n\nYou', 'H', 'A\n\nYou', 'B', 'e\n\nTo', 'F\n\nYou', '_ _ _', 'S\n\nYou', 'A\n\nYou', 'T\n\nYou', 'Q\n\nYou', 'S\n\nYou', '_ _ _', 'U', 'G\n\nYou', '_ a g', '_ e r', '_ a r', '_ a _', '_ _ _', 'W', 'L\n\nYou', '_ _ _', 'm\n\nYou', 'E', '_ n k', 'p\n\nYou', '_ m p', 'J\n\nThe', 'a\n\nYou', '_ _ _', '_ r u', 'Q\n\nYou', 'T', 't\n\nThe', 'F', '_ p _', '_ _ t', 'H', '_ _ _', 'n\n\nYou', 'a\n\nTo', 'h\n\nThe', 'O\n\nYou', 'a\n\nThe', 'N', 'E\n\nYou', '_ e e', 'N\n\nYou', 'E', 'T', 'F\n\nYou', 'MDD', 'F\n\nYou', '_ f t', 'X\n\nYou', '_ _ _', '_ _ _', '_ _ _', '_ _ _', 'p', 'r\n\nThe', 'c\n\nThe', 'c\n\nThe', 'N', 'c\n\nThe', 'A', 'E', 'Q\n\nYou', 'A\n\nYou', 'C\n\nYou', 'H\n\nYou', '_ x i', 'E\n\nYou', 'QN', 'H\n\nYou', 'c\n\nYou', 's\n\nThe', '_ _ _', 'c\n\nThe', 's\n\nThe', 'H', 'Y\n\nYou', 'E\n\nYou', 'W\n\nYou', 'O', 'E\n\nYou', 'W\n\nYou', 'p', 'U', 'U', 'p', '_ _ _', 'c', 'c\n\nYou', 'r\n\nThe', 'r\n\nThe', '_ _ _', 'P', '_ _ i', 'h\n\nYou', 'P', 'C', 'H', 'D\n\nYou', 'S\n\nYou', 'G\n\nYou', 'r\n\nYou', 'E\n\nYou', 'H\n\nYou', 'h\n\nYou', '_ t _', 't\n\nThe', 't', '_ _ _', 'K\n\nYou', 'A\n\nYou', 'F\n\nYou', 'B', 'S\n\nYou', '_ i _', 'WSL', 'G\n\nYou', 'K\n\nYou', 'I\n\nYou', 'M\n\nYou', 'S\n\nYou', 'K\n\nYou', 'n\n\nThe', '_ _ _', 'c', 'V', 'c\n\nYou', 'r\n\nYou', 'R', 'd\n\nYou', 'T\n\nThe', 'B', 'r\n\nThe', 'I\n\nYou', 'X\n\nThe', 'D\n\nYou', 'J\n\nYou', 'M\n\nYou', '_ _ _', 'F\n\nYou', 'A', '_ _ _', 'm\n\nYou', 'A', 'U', 'U', 'L\n\nThe', 'n\n\nThe', '_ _ _', 'F\n\nThe', 'P\n\nYou', '_ _ c', 'a\n\nYou', '_ _ _', 'F\n\nYou', 'F\n\nYou', 'A', '_ _ _', 's t r', 'V', 'A', '_ _ r', 'G', 'A\n\nYou', 'A\n\nYou', 'H\n\nYou', 'A\n\nYou', 'H\n\nYou', 'p\n\nThe', 'O', '_ o p', 'H\n\nYou', '_ _ _', 'P', "'I'", 'R\n\nYou', 'r\n\nThe', 'U', '_ _ _', 'r\n\nThe', 'r\n\nThe', '_ e t', '_ i _', 'W\n\nYou', '_ _ n', 'R\n\nYou', 'Z\n\nYou', '_ _ n', 'W', 'YC', 'G\n\nYou', 'J', 'G', 'R', 'R\n\nYou', 'a\n\nTo', 'QFX', 'J\n\nYou', 'T', 'H\n\nYou', 'R\n\nYou', 'T\n\nYou', 'a\n\nYou', '_ _ a', 'J\n\nYou', 'C\n\nYou', 'N', 'n', '_ y _', '_ _ _', '_ _ _', 'O\n\nYou', 'U', '_ u b', 't\n\nThe', 'r\n\nThe', '_ _ _', 'e\n\nThe', '_ i _', 'J\n\nYou', '_ _ n', '_ _ _', 'P\n\nYou', '_ _ _', '_ _ _', 'U', 'C\n\nYou', '_ _ _', 'Q\n\nYou', 'u\n\nYou', 'E\n\nYou', '_ a s', 't _ s', 'Q\n\nYou', 'd\n\nThe', '_ t _', '_ t u', 'p\n\nTo', '_ t u', 's p e', '_ u d', 'd\n\nThe', 'd\n\nYou', '_ u _', '_ u _', 'L', '_ _ _', 'E\n\nYou', 'c', 'H', 'W', 'A', 'T\n\nYou', 'e _ _', 'W', 'A\n\nYou', 'n', 's _ _', 'K', '_ c a', '`A`', 'r\n\nYou', 'M\n\nYou', 'A', 'r\n\nYou', 'A\n\nYou', 't\n\nThe', 'r\n\nThe', 'E', 'f\n\nYou', '_ e f', 'r\n\nYou', 'FVRD', 'O\n\nYou', 'ZP', '_ e a', 'U\n\nYou', 'A\n\nYou', 'T\n\nYou', 'A', '_ _ _', 'F\n\nThe', 'M', 'H\n\nTo', 'W\n\nThe', 'R', 'Y\n\nYou', 'A\n\nYou', 'W', '_ a l', 'M\n\nYou', '_ _ l', '_ a l', '_ i r', 'O\n\nYou', 'JRT', 'Q\n\nYou', 'R', '_ _ r', '_ _ a', 'r\n\nThe', 't', '_ _ _', 'U', 'S\n\nYou', 'Q\n\nYou', 'F\n\nYou', 'A', 'p\n\nThe', 'p\n\nYou', '_ _ _', 'Q\n\nYou', 'r\n\nThe', 'G', 'm\n\nThe', 'M\n\nYou', 'r\n\nYou', 'r\n\nYou', 'J\n\nYou', 'K', '_ _ s', 'P', 'H', 'T\n\nYou', 'T\n\nYou', '_ _ _', 'Q\n\nYou', 'F\n\nYou', 'K\n\nYou', 'X\n\nYou', 'M\n\nYou', '_ _ _', 'F\n\nYou', 'M\n\nYou', 'KTN', 'K\n\nYou', 'k\n\nYou', 'A', 'F', 'H\n\nYou', '_ _ s', 'r\n\nThe', 'r\n\nYou', 'FRY', 'K', 'P', 'YSS', '_ e s', 'D', 'S\n\nYou', 'ZV', 't\n\nYou', 'Z\n\nYou', 'K', 'K', 'G\n\nYou', 'H', 'U\n\nYou', 'O\n\nYou', 'U', 'c\n\nYou', '_ _ _', 'N\n\nYou', 'H\n\nYou', 'F', 't _ i', 'H', 'B', 'T\n\nYou', 'W\n\nYou', '_ s _', 'E', 'K\n\nYou', 'e\n\nTo', 'S\n\nYou', 'y\n\nYou', 'u\n\nYou', 'c d l', 'R\n\nYou', 'F\n\nYou', '_ o _', 'H\n\nYou', 'O', 'H', 'H', 'U\n\nYou', '_ e g', 'r\n\nYou', '_ _ _', 'R\n\nYou', 'O', 'L\n\nYou', 's\n\nYou', '_ r _', 'r\n\nYou', 'r\n\nYou', '_ _ l', 'Z\n\nThe', '_ _ _', '_ e _', '`p`', 'E\n\nYou', 'R', 'R\n\nYou', 't\n\nThe', 'D\n\nYou', 'Q\n\nYou', 'X\n\nThe', 'r\n\nYou', 'E\n\nYou', 'F\n\nYou', 'N', 'H\n\nYou', 'n', 'r\n\nThe', 'd', 'd\n\nThe', 'r\n\nThe', '_ _ i', 'N', 'T', 'G', 'P\n\nYou', 'W', 'bwny', 'X\n\nYou', 'N', 'A', 'R', 'J\n\nYou', '_ _ i', 'H', 'M\n\nThe', 'u\n\nTo', '_ l a', '_ _ _', '_ l a', 'E\n\nYou', 'RQ', 'L', 'M\n\nYou', 'i\n\nThe', '_ l a', 'M\n\nYou', 'F\n\nYou', 'A', 'F\n\nYou', 't\n\nThe', 't\n\nYou', 'A', 'A', '_ _ _', 'I\n\nYou', 'Y\n\nYou', '_ _ _', '_ o _', 'JCGO', '_ _ i', 'h\n\nTo', 'F\n\nTo', 'E\n\nYou', 'e\n\nYou', 'P', '_ a l', '_ _ i', 'I\n\nYou', 'Y\n\nYou', 'r\n\nYou', 'C\n\nYou', 'X\n\nYou', 'A\n\nYou', 'M', 'A\n\nYou', 'U\n\nYou', 't o o', 't\n\nThe', 'T', 'h\n\nTo', 't _ _', 'A', 'O', 'A', 'e', 'i', 'd', 'R\n\nYou', 'd', '_ _ _', 'q\n\nThe', 'r\n\nThe', 'A\n\nYou', 'C\n\nYou', '_ i _', 'K\n\nYou', 'G', 'W', 'W\n\nYou', 'r\n\nYou', 'A', 'A', 'r\n\nThe', 'r\n\nYou', 'J\n\nYou', 't r r', 'T\n\nYou', '_ d _', '_ _ o', 'G', 'r\n\nThe', 'R']

In [ ]:
for c in letter_preds:
  print(c)

In [18]:
is_single_letter = [len(pred) == 1 and pred.isalpha() for pred in letter_preds]
# print(is_single_letter)
print(letter_preds)
print(f"Number of single letters: {sum(is_single_letter)}")

['X', '_', 'H', 'p', 'i', 'A', 'A', 'V', 'p', 't', '_', 'H', 'J', 'A', 'A', 'A', '_', 'Q', 'y', '_', 'U', 'F', 'C', 'H', '_', '_', 'f', 'A', 't', '_', 't', 'r', 't', 'H', 'A', 's', 'a', 'D', 'J', '_', 'h', '_', 'W', 'H', 'G', 'I', 'n', 'J', 'B', 'n', 'J', 'A', '_', 'A', '_', 'm', '_', '_', 'o', 'c', 'H', 'A', 'B', '_', 'Z', 'I', 'c', 'c', 'E', 'c', 'U', '_', 'A', 'M', 'W', 'D', 'r', 'Y', '_', 'p', 'r', 'F', 'S', 'R', 'R', 'n', 'T', 'L', 'Q', 'U', 'A', '_', 'S', 'A', 'A', 'I', 'J', '_', 'P', 'Q', 'F', '_', 'Z', 'H', '_', 'r', 'L', 't', 't', 'p', 'V', 'p', 'p', 'T', 'W', '_', 'Q', 'W', 'B', 'H', 'l', 'Z', 'H', 'h', 'A', '_', '_', '_', '_', 'P', '_', '_', 'q', 't', 'U', 'A', '_', 'V', 'R', 'Z', '_', 'G', 'R', 'I', '_', '_', '_', '_', '_', '_', 'B', 'Q', 'V', '_', 'S', 'K', 'K', 'F', 'K', 'E', 'F', 't', 'A', 't', 't', 'P', 'F', 'A', 'a', 'a', 'X', '_', 'K', '_', 'A', '_', 'J', 't', 'X', 't', 'm', 'N', '_', '_', 'N', 'z', '_', 'H', 'R', 'A', 'A', 'A', '_', 'W', 't', 't', 't', 't', '_', 'H',

##Output

In [1]:
output_letter = ['X', '_', 'H', 'p', 'i', 'A', 'A', 'V', 'p', 't', '_', 'H', 'J', 'A', 'A', 'A', '_', 'Q', 'y', '_', 'U', 'F', 'C', 'H', '_', '_', 'f', 'A', 't', '_', 't', 'r', 't', 'H', 'A', 's', 'a', 'D', 'J', '_', 'h', '_', 'W', 'H', 'G', 'I', 'n', 'J', 'B', 'n', 'J', 'A', '_', 'A', '_', 'm', '_', '_', 'o', 'c', 'H', 'A', 'B', '_', 'Z', 'I', 'c', 'c', 'E', 'c', 'U', '_', 'A', 'M', 'W', 'D', 'r', 'Y', '_', 'p', 'r', 'F', 'S', 'R', 'R', 'n', 'T', 'L', 'Q', 'U', 'A', '_', 'S', 'A', 'A', 'I', 'J', '_', 'P', 'Q', 'F', '_', 'Z', 'H', '_', 'r', 'L', 't', 't', 'p', 'V', 'p', 'p', 'T', 'W', '_', 'Q', 'W', 'B', 'H', 'l', 'Z', 'H', 'h', 'A', '_', '_', '_', '_', 'P', '_', '_', 'q', 't', 'U', 'A', '_', 'V', 'R', 'Z', '_', 'G', 'R', 'I', '_', '_', '_', '_', '_', '_', 'B', 'Q', 'V', '_', 'S', 'K', 'K', 'F', 'K', 'E', 'F', 't', 'A', 't', 't', 'P', 'F', 'A', 'a', 'a', 'X', '_', 'K', '_', 'A', '_', 'J', 't', 'X', 't', 'm', 'N', '_', '_', 'N', 'z', '_', 'H', 'R', 'A', 'A', 'A', '_', 'W', 't', 't', 't', 't', '_', 'H', 'h', 'A', 'H', 'C', 'A', 'U', 'A', 'F', 'U', 'I', 'A', 'M', 'T', 'M', '_', '_', 'E', 'U', 'U', 'U', 'H', 'U', 't', 'b', 'R', 'C', 'R', 'E', '_', 'R', 'C', 'T', 'c', 'E', '`', 'B', 'H', '_', 'B', 'E', 'A', 'E', 'X', 'E', 'U', 'A', '_', '_', 'V', '_', '_', 'W', 'E', 'H', 't', '_', 'Z', 'U', 's', 'A', 'G', 't', 't', '_', 'H', 't', 'D', 'U', '_', 'N', 'G', '_', 'A', 'A', '_', '_', 'J', 'H', 'A', 'A', 'T', 'A', '_', 'm', 'N', 'J', '_', 'A', '_', 't', '_', 'R', '_', '_', 'W', 'G', 'J', 'A', 'F', 'E', 'd', '_', 'Z', 'J', '_', 'U', 'd', '_', 'A', '_', '_', 'L', 'F', 'O', 'C', 'A', 'F', 'D', 'K', 'N', 'Z', 'r', '_', 'r', '_', 'T', 'X', 't', 'X', '_', 't', '_', 'E', '_', 'p', 'p', 'W', 'W', 'H', 'H', 'R', 'H', 'V', 'R', 'A', 'J', '_', 'M', 'm', '_', 'E', '_', 'H', 'V', 'F', 'F', 'F', 'r', 'F', 'U', 'R', 'H', 'G', 'E', 'O', 'i', 'G', 'B', 'X', 'A', 'N', 'K', '_', 'Y', 'A', 't', 'c', 'K', 'T', '_', 'P', '_', 'X', '_', 'r', 't', 'H', 'Z', 'X', 'E', 'd', 'O', 'X', 'i', 'L', 'S', 'W', '_', 'U', 'O', 'V', 'T', 'R', '_', 'I', '_', 'K', 'T', 'T', 'Q', 'B', 'L', 'Q', 'Y', 'R', 'O', 'Z', 'W', 'l', 'W', '_', '_', 'V', 'D', 'D', 'W', 'd', '_', '_', 'J', 'O', '_', 'P', 'H', 'M', '_', 'c', 'c', 'X', 'M', '_', 'I', 'I', 'X', 'U', 'h', '_', 'C', 'i', '_', 'r', '_', 'V', '`', '`', '_', '_', 'F', 'F', '_', '_', 'N', 'O', 'O', 'H', '_', 'c', 'n', '_', 'l', 'H', 'F', '_', 'F', 'L', 'Y', 'Q', 'A', 'M', 's', '_', 'd', 'o', 'H', 'F', 'O', 'l', '_', 'Q', '_', 'h', '_', 'h', 't', '_', 'y', 'T', '_', 'A', 'O', 'T', '_', 'A', 'L', 't', 'h', 'W', 'H', 'O', 'r', 'H', 'H', 'H', 'D', 'H', 'J', 'J', 'W', 'N', 'A', '_', 'X', 'e', 'E', 'M', '_', 'W', 'U', 'A', 'P', 'M', 'm', 'r', 'H', 'n', 'r', 'H', 'A', 'B', 'e', 'F', '_', 'S', 'A', 'T', 'Q', 'S', '_', 'U', 'G', '_', '_', '_', '_', '_', 'W', 'L', '_', 'm', 'E', '_', 'p', '_', 'J', 'a', '_', '_', 'Q', 'T', 't', 'F', '_', '_', 'H', '_', 'n', 'a', 'h', 'O', 'a', 'N', 'E', '_', 'N', 'E', 'T', 'F', 'M', 'F', '_', 'X', '_', '_', '_', '_', 'p', 'r', 'c', 'c', 'N', 'c', 'A', 'E', 'Q', 'A', 'C', 'H', '_', 'E', 'Q', 'H', 'c', 's', '_', 'c', 's', 'H', 'Y', 'E', 'W', 'O', 'E', 'W', 'p', 'U', 'U', 'p', '_', 'c', 'c', 'r', 'r', '_', 'P', '_', 'h', 'P', 'C', 'H', 'D', 'S', 'G', 'r', 'E', 'H', 'h', '_', 't', 't', '_', 'K', 'A', 'F', 'B', 'S', '_', 'W', 'G', 'K', 'I', 'M', 'S', 'K', 'n', '_', 'c', 'V', 'c', 'r', 'R', 'd', 'T', 'B', 'r', 'I', 'X', 'D', 'J', 'M', '_', 'F', 'A', '_', 'm', 'A', 'U', 'U', 'L', 'n', '_', 'F', 'P', '_', 'a', '_', 'F', 'F', 'A', '_', 's', 'V', 'A', '_', 'G', 'A', 'A', 'H', 'A', 'H', 'p', 'O', '_', 'H', '_', 'P', "'", 'R', 'r', 'U', '_', 'r', 'r', '_', '_', 'W', '_', 'R', 'Z', '_', 'W', 'Y', 'G', 'J', 'G', 'R', 'R', 'a', 'Q', 'J', 'T', 'H', 'R', 'T', 'a', '_', 'J', 'C', 'N', 'n', '_', '_', '_', 'O', 'U', '_', 't', 'r', '_', 'e', '_', 'J', '_', '_', 'P', '_', '_', 'U', 'C', '_', 'Q', 'u', 'E', '_', 't', 'Q', 'd', '_', '_', 'p', '_', 's', '_', 'd', 'd', '_', '_', 'L', '_', 'E', 'c', 'H', 'W', 'A', 'T', 'e', 'W', 'A', 'n', 's', 'K', '_', '`', 'r', 'M', 'A', 'r', 'A', 't', 'r', 'E', 'f', '_', 'r', 'F', 'O', 'Z', '_', 'U', 'A', 'T', 'A', '_', 'F', 'M', 'H', 'W', 'R', 'Y', 'A', 'W', '_', 'M', '_', '_', '_', 'O', 'J', 'Q', 'R', '_', '_', 'r', 't', '_', 'U', 'S', 'Q', 'F', 'A', 'p', 'p', '_', 'Q', 'r', 'G', 'm', 'M', 'r', 'r', 'J', 'K', '_', 'P', 'H', 'T', 'T', '_', 'Q', 'F', 'K', 'X', 'M', '_', 'F', 'M', 'K', 'K', 'k', 'A', 'F', 'H', '_', 'r', 'r', 'F', 'K', 'P', 'Y', '_', 'D', 'S', 'Z', 't', 'Z', 'K', 'K', 'G', 'H', 'U', 'O', 'U', 'c', '_', 'N', 'H', 'F', 't', 'H', 'B', 'T', 'W', '_', 'E', 'K', 'e', 'S', 'y', 'u', 'c', 'R', 'F', '_', 'H', 'O', 'H', 'H', 'U', '_', 'r', '_', 'R', 'O', 'L', 's', '_', 'r', 'r', '_', 'Z', '_', '_', '`', 'E', 'R', 'R', 't', 'D', 'Q', 'X', 'r', 'E', 'F', 'N', 'H', 'n', 'r', 'd', 'd', 'r', '_', 'N', 'T', 'G', 'P', 'W', 'b', 'X', 'N', 'A', 'R', 'J', '_', 'H', 'M', 'u', '_', '_', '_', 'E', 'R', 'L', 'M', 'i', '_', 'M', 'F', 'A', 'F', 't', 't', 'A', 'A', '_', 'I', 'Y', '_', '_', 'J', '_', 'h', 'F', 'E', 'e', 'P', '_', '_', 'I', 'Y', 'r', 'C', 'X', 'A', 'M', 'A', 'U', 't', 't', 'T', 'h', 't', 'A', 'O', 'A', 'e', 'i', 'd', 'R', 'd', '_', 'q', 'r', 'A', 'C', '_', 'K', 'G', 'W', 'W', 'r', 'A', 'A', 'r', 'r', 'J', 't', 'T', '_', '_', 'G', 'r', 'R', 't', 'r', '_', 't', '_', 'X', '_', '_', 'M', 'G', '_', 'V', 's', 'G', 'B', 'Z', 'R', 'P', 'r', 'K', 'R', 'T', 'J', 'e', 'e', 'B', '_', '_', 'h', 'h', 'o', 'D', 'H', 'w', 'S', 'E', 'E', 'E', 'E', '_', 'A', 'r', 'r', 'r', 'r', 'F', '_', '_', '_', 's', 'r', 'J', 'F', 'J', 'M', 'F', '_', '_', 'P', 'U', 'U', 'F', 'D', '_', 'U', 'M', 'H', '_', 'B', 'D', 'Q', 'E', 'A', 'r', 'R', 'S', 'S', 'r', 'm', 'A', '_', 'n', 'M', 'T', 'X', 'L', 'd', 'r', 'T', 'S', 'M', 'N', 'A', 'H', 'F', '_', '_', 'X', '_', 'J', 'J', '_', 'o', 'M', 'b', 'X', 'X', 'W', '_', 'F', '_', 'U', 's', 'K', '_', '_', 'O', 'h', 'r', '_', 'H', 'O', 'U', 'A', '_', '_', 'U', '_', 'A', 'd', 'B', 'S', 'd', 's', '_', 'M', 'U', 'U', 'U', '_', 'T', 'e', 'M', 't', 't', 'Q', 't', 'U', 'B', 'M', '_', 'A', 'J', 'n', '_', 'R', 'E', 'X', 'T', 'e', 'X', '_', 'O', 'H', 'E', 'E', '_', '_', 'a', 'A', 'a', 'A', 'B', 'X', 'G', 'H', '_', 'F', 'C', 'o', '_', '_', '_', '_', 'T', 'Y', 'E', 'D', 'R', 'A', 'E', 'D', 'T', 'G', 'S', 'p', '_', 'r', 'S', 'F', 'D', 'F', 'd', 'r', 'W', 'G', 'E', 'L', 'B', 'c', '_', '_', '_', 'p', '_', 'Z', '_', 'U', 'W', 'U', 'W', 't', 'P', 'M', '_', 'O', 'p', 's', 'Y', 'R', 'X', 'S', 'I', '_', 'D', '_', 'F', '_', 'a', 'W', 'R', 'R', 'R', 'M', 'A', 'T', 'r', 'C', 'E', '_', '_', '_', '_', '_', 'Y', 'c', '_', 'a', 'A', 'A', 'E', 'V', '_', '_', 'M', 'G', '_', '_', 'Q', 'M', 'p', 'W', '_', 'E', 'r', '_', 'H', 'h', 'R', 's', 'c', 'o', 'c', 'T', 'h', 'U', 'M', 'Q', 'N', 'M', 'A', '_', '_', '_', '_', 'H', 'F', 'U', 'H', 'r', 'W', 'P', 'R', 'A', '_', '_', 'R', 'r', '_', 'G', 'r', 'P', 'F', 'F', '_', 't', '_', 't', 'C', 'H', 'H', 'W', 'R', 'M', 'h', 'J', 'R', 'B', 'P', 'Z', 'S', 't', '_', 'Q', 'N', 'J', '_', 'P', 'S', 'R', 'P', 'm', 'F', 'O', 'D', 'R', 'H', '_', 'H', 'K', '_', 'p', 'Z', 'A', 'V', 'K', '_', 'B', 'R', 'a', 'K', 'b', 'E', '_', 'H', 't', '_', 'E', 'C', 'A', 'N', 'C', 'A', 'I', 'W', 'A', 'c', '`', 't', 'A', '`', 'o', '_', 'B', 'G', 'P', 'a', 's', '_', 'h', '_', 'H', '_', 'N', 'T', 'P', 'W', 'E', '_', 'E', 'T', 'H', 'J', 'p', 'T', 'd', 'd', 'p', 'U', 'm', 'r', 'p', 'E', 'P', 'r', 'N', '_', 'M', 'B', 'O', '_', 'N', 'S', 'K', '_', 'r', '_', '_', 'R', 'N', '_', '_', 'K', 'A', '_', '_', 'R', 'r', 't', '_', 'N', 'm', '_', 'N', 'K', '_', 'A', 'E', 'r', 'W', 'a', 'V', '_', '_', '_', 'T', '_', 'Q', 'X', 'H', 's', 'A', 'H', 'h', 'C', 'A', 'N', 'C', 'V', 'h', 'h', 'R', 'h', 'Q', 'R', '_', '_', '_', 'N', '_', 'Q', 'T', 'E', 'V', 'A', 'A', 'O', 't', 'H', '_', 'Z', 'N', 'a', 'd', 'A', '_', 'X', 'H', '_', 'T', 'H', 'J', '_', 'G', 'H', 'D', '_', 'V', 'D', 'V', '_', '_', 'O', 't', '_', '_', 't', 'B', '_', 's', 'F', '_', 'r', 't', 'L', '_', 'M', 'A', '_', '_', '_', '_', 'm', 'n', 'd', 'd', 'r', 'P', 'c', 'H', 'r', 'S', 'H', 'c', 'Q', 'l', '_', '_', 'n', 'N', 'H', 'H', 'e', 'u', 'u', '_', '_', 'Z', '_', 'o', 'H', 'N', 'V', 'O', '_', 'F', '_', 'r', 'P', 'H', '_', 'K', 'F', 'k', 'K', 'K', 'U', 'U', 'R', 'E', '_', 'G', '_', 'A', 'O', 'F', 'r', '_', 'H', 'G', 'Q', 'J', 'J', 'd', 'K', 'U', 'b', 'N', 'S', 'U', 'c', 'W', '_', 't', 'p', 'P', 'p', 't', 'U', 't', 'e', 'p', 'M', 'X', 'W', 'H', 'I', 'P', 'R', 'w', 'K', 'R', 'O', 'P', 'R', '_', '_', '_', 'K', '_', 'c', '_', '_', 'r', '_', 'E', 'D', 'u', 'A', '_', 'p', 'P', 'H', 'N', 'r', 'J', 'H', '_', 'M', '_', '_', 'c', 'C', '_', 'M', 'c', 'c', 'F', 'M', '_', 'R', 'R', 'M', 'P', '_', '_', 'L', 's', 'Q', '_', 'N', 'i', 'J', 'A', '_', 'U', '_', 'T', 'Q', '_', 'U', '_', 'N', 't', 'A', '_', '_', 'X', 'Q', 'L', 'p', '_', '_', 'm', 's', 't', '_', 'G', 'S', 'E', 'G', 'G', 'N', 'r', 'H', 'P', 'r', '_', 'o', 'G', 'I', 'Z', 'V', '_', 'A', 'N', '_', 'c', '_', 'n', 'Y', 'L', '_', 'd', 'c', 'M', 'O', 'V', '_', 's', '_', '_', 'A', 'A', 'A', 'C', 'i', 'Z', 'K', '_', 'V', 'E', 'r', '_', '_', 'Z', 'K', 'c', 'c', 'h', 'h', 'B', '_', '_', 'Q', 'W', '_', '_', '_', 'M', 'd', '_', 'E', 'U', 'r', 'O', '_', 'F', '_', 'M', 'W', 'N', '_', 'M', 'T', 'M', 'R', 'E', 'A', 'F', '_', 'E', 'c', '_', 'P', 'r', 'A', 'A', 'A', 'R', 'H', 'A', 'H', 'S', 'X', 's', 'o', 'G', 'X', 'e', 'P', 'Q', '_', '_', 'J', '_', 'm', '_', 'K', '_', 'k', 'O', 'B', '_', 'F', 'H', '_', 'R', 't', 'F', 'p', '_', 't', '_', 'A', 'T', 'd', 'H', 'U', 'E', 'J', 'i', 's', 'A', 'W', 'P', '_', 's', 'c', 'C', 'n', 'c', 'O', 'L', 'X', 'M', 'M', 's', 'F', 'R', '_', 's', 'E', 'P', '_', '_', 'A', 'K', 'k', '_', 'A', 'M', 'r', 'B', '_', 'h', 'T', '_', 't', 'P', 'J', 'F', 'N', '`', '_', 'E', '_', 'H', 'l', 'c', 'A', 'h', 'W', '_', '_', '_', 'Q', '_', '_', '_', 'Q', 'F', 'D', '_', 'B', 'K', 'K', 's', 'E', '_', 'K', 't', '_', '_', 't', 'P', 'B', 'X', 'W', 'c', 't', 'e', 'r', 'h', 'H', 'r', 'E', 'p', '_', 'e', 's', 'N', '_', 'M', 'A', 'r', 't', 'U', 'A', '_', 'r', 'P', '_', 'A', 'T', '_', 'O', 'F', 'z', '_', 'B', 'z', 'F', 'M', 'Z', 'M', '_', 'h', 'F', 'J', 'W', '_', 'Z', 's', '_', 'U', 'Q', 'J', 'X', 'W', '_', 'F', 'V', 'W', 'P', 'w', '_', 'K', '_', 'S', 'T', 'S', 'I', 'w', 'W', '_', 'T', '_', 'Z', 'W', 'H', 's', 't', 'W', 'a', 'r', 'r', 'p', 'M', 'a', 'A', 'D', 'B', 'm', 'W', 's', 'S', 'C', '_', 'K', 'P', 'h', 'F', 'T', 'H', '_', 'h', '_', 'a', '_', '_', 'R', 'S', '_', '_', 'P', 'W', 'E', 'M', 'G', 'H', '_', 'A', 'a', 'O', 'X', 'Z', 'X', '_', 'C', 'W', '_', 'U', 'A', 'a', 'N', 't', '_', 'A', 'Y', 'V', '_', 'p', 'W', 'K', 'G', 'Z', 'W', '_', 'h', '_', 'E', 'H', 'h', '_', '_', 'P', '_', '_', 'M', 'M', 'a', '_', 'A', 'A', 'X', 'r', 'e', '_', 'N', 'd', '_', 'A', 'D', '_', '_', 'R', 'R', 'A', 'O', 'O', 'H', 'w', 'r', '_', '_', 'F', 'F', 'F', '_', 'E', 'E', 'A', 'K', 'E', 'm', 't', '_', '_', 'W', 'E', '_', 'J', 'p', 'E', 'e', '_', '_', '_', 'R', 'U', '_', 'L', 'A', '_', 'A', '_', '_', 't', 'W', 'W', 'Q', 'S', 'L', 'F', '_', 'R', 'D', 'V', 'U', 'B', 'H', 'R', 'h', '_', 'X', 'E', 'T', 's', 'H', 'H', 'F', '_', 'A', 'S', '_', 'I', 'G', 'K', 'd', 'U', 'A', 'A', 'E', 'o', 'F', 'm', 'U', '_', 'Y', 'd', 'A', 'O', 'A', '_', '_', 't', 'r', 'A', 'W', 'L', 'W', '_', 'M', 'K', 'H', 'H', 'M', 'r', 'R', 't', 'K', 'H', 'Q', '_', 'A', 'I', '_', 'w', 'w', 'w', 'F', 'E', 'N', 'n', 'Y', 'H', 'U', 'A', 'U', '_', 't', 'b', '_', '_', 't', '_', '_', 'E', 'U', 'c', 'J', 'J', '_', 'J', 'R', 'J', 'd', '_', 'J', 'u', 'R', 't', 't', 'o', 'U', 'G', 'c', 'g', 'Q', 'L', 'Z', 'H', '_', 'T', 'c', '_', '_', 'Q', 'U', 'Z', '_', 'e', 'H', '_', 'b', 'B', 'U', 'r', 'A', 'J', 'Z', 'V', 'A', 'l', '_', 'D', '_', 'O', '_', 'F', 'W', 'H', 'W', 'E', 'r', 'c', 'U', 'H', '_', 'N', 'E', 'W', 'h', 'c', 'R', 'O', '_', 'H', 'J', 'E', '_', 'r', 'F', 'Z', 't', 'c', '_', 'E', 'W', '_', 'm', 'm', 'O', 'm', 'A', 'u', 'E', 'U', 'u', 'r', 'I', 'P', 'P', 'N', '_', 'L', 'P', 'E', 'H', '_', 'H', 'H', '_', 'S', 'h', '_', '_', 'n', 'E', '_', 'W', '_', 'U', 'W', 'e', '_', 'r', 'R', '_', 'm', 'c', 'P', 'p', 'U', '`', 'p', '_', 'B', 'p', '_', 'p', 'W', 'Q', 'Q', 'E', 'p', '_', '_', '_', 'U', 'K', 'B', 'P', 'V', 'r', '_', '_', 'D', 'n', 's', 'J', 'A', 'M', 's', 'C', 'E', 'B', 'S', 'T', 'X', 'A', 'R', '_', 'K', 'F', '_', 'c', '_', 'M', 'r', '_', 'E', 'c', 'A', '_', 'F', 'W', '_', 's', 'M', '_', '_', 'P', 'i', 'W', '_', '_', '_', 'w', '_', 'w', 'C', 'M', 't', 'K', '_', '_', 't', 's', '_', 'W', 'K', '_', 'A', 'n', 'd', 'r', 'X', '_', '_', 'O', '_', 'A', 'E', 'X', 'S', 'A', '_', 'O', 'M', 'U', '_', 'A', 'E', 'B', 'p', 'r', '_', 'A', 'c', 'F', 'A', 'M', 'c', 't', 'c', 'U', '_', 'A', 'K', 'O', '_', 'J', 'Z', '_', 'L', 'R', 'J', 'R', 'M', 'c', 'd', '_', 'N', 'Z', 'C', 'U', 'F', '_', 'c', 'U', 'c', 'H', 'L', 'W', 'V', 'X', '_', 'U', 'H', '_', 'O', 'K', '_', '_', '_', 'u', 'W', 'U', 'F', '_', 'M', 'X', '_', 'U', 'A', 'H', 'm', 'M', 'H', 'A', 'U', '_', 'a', 't', 'F', '_', 't', 'w', 'w', 'r', 't', 't', '_', 'O', 'F', '_', 'V', 'H', 'h', 'h', 'i', 'G', '_', 'd', '_', 'H', 'J', 'M', 'V', 'T', '_', 'E', '_', '_', '_', '_', 'X', 'X', 'E', 'V', 'M', 'P', 'h', 'B', 'R', 'U', 'q', 'D', 'A', 'Q', 'c', 'L', 'A', 'A', 'c', 'G', 'G', 'E', 'Z', 'J', 'c', 'c', 'c', 'N', 'A', 'H', 'M', 'R', 'A', 'c', 'r', '_', '_', '_', 'G', 'G', 'K', '_', 'r', '_', '_', '_', 'E', 'H', 'E', 'R', '_', '_', 'J', '_', 'G', '_', 'A', '_', 'a', '_', 'P', 'H', 't', '_', 't', 'h', 'E', 'A', '_', 'R', 's', 'E', 'R', 'F', '_', 'H', 'Z', 'X', 'O', 'E', 'E', 'c', 'c', 'u', 'n', 'H', 'U', '`', 'Q', '_', '_', 'A', 'H', 'M', '_', '_', '_', 'c', 'K', 'J', 'F', 'a', 'F', 'J', 'p', 'U', '_', 'U', 'p', '_', 'w', 'F', 'A', '_', '_', 'p', 'F', 'W', 'X', 'M', 'A', 'V', '_', 'm', 'A', 'P', 't', 'L', 't', 'M', 't', 'F', 'A', '_', 'A', '_', 'd', 'V', 'J', 'Z', 'E', 'B', 'c', 'K', 't', 'P', 'a', 'R', 's', 't', 's', 's', 's', 'U', 'A', 'A', '_', 'C', '_', 'J', '_', 'p', 'G', 'V', 'C', 'V', 'V', '_', 'n', '_', '_', 'i', 'Y', 'i', 'i', '_', 'r', '_', 'U', 'H', 'U', 'T', '_', 'c', 'M', 'Q', 'P', 'H', 'F', 'A', 'A', 'O', 'Q', 'P', '_', 'H', 'O', 'Y', '_', '_', 'H', 'p', 'D', 'r', 'J', 'T', '_', '_', '_', 'r', 'Q', '_', '_', '_', 't', 's', '_', 'd', 'B', 'P', '`', 'c', 'M', 'R', 'E', 'A', 'Q', 'V', '_', '_', '_', 'U', 'G', 'E', 'T', 'M', '_', 'V', 'W', '_', '_', 'Q', 'K', 'E', 'E', 's', 'V', 's', 'K', 's', 'C', 's', 'D', '_', '_', 's', 'W', 'W', 'H', 'N', 'A', '_', '_', '_', 'W', '_', '_', 'r', 'H', 'Z', 'Y', 'R', '_', 'V', '_', 'U', 'E', 'h', 'H', 'H', 'K', 's', 'u', '_', 't', 't', '_', 't', 'Q', '_', 'r', '_', '_', '_', 'C', '_', '_', 'g', 'P', 'F', 'X', 'J', 'F', 'H', 'Z', 'r', '_', '_', '_', '_', 'Q', 'N', '_', 'T', 'c', '_', '_', 'Q', '_', '_', 't', '_', 'H', 't', 'R', 'l', 'R', 'T', 'B', 'R', 'w', 'w', 'w', 'w', 'B', '_', 'A', 'A', 't', '_', '`', 'W', '_', 'U', 'E', 'G', '`', 'F', 'E', 'F', 'X', 'U', 'F', 'A', 'D', 'C', 'n', '_', 'N', 'H', 'K', 'Q', 'L', 'P', 'J', '_', 'U', 'N', 'Q', '_', 'N', 'M', 'D', '_', 'c', 's', 'E', 'E', 'M', 't', 'Y', 'T', 'E', 'P', 'M', 'U', 'F', 'F', '_', 'P', 'i', 'p', 'M', 'F', '_', 'A', '_', 'L', 't', '_', '_', '_', '_', 'Q', 'P', 'R', 'G', 'I', 'H', 'A', '_', 'h', '_', 'p', 'X', 'p', '_', '_', '_', '_', 'W', 'A', 'Q', 't', 'e', 'N', '_', 'T', '`', 'J', '_', 'H', '_', 'c', 'A', 'c', 'h', 'H', 'U', 'W', 'B', 'N', 'P', '_', 'R', '_', 'Z', 'I', 'M', 'e', 's', 'M', 's', 's', 'a', 'U', 'H', '_', 'H', 'T', 'B', 'I', '_', 'A', 'A', 'U', '_', 'A', 'A', 'A', '_', 'G', 'A', 'R', '_', 'A', '_', 'H', 'Z', 'A', '_', 'H', 'O', '_', '_', 'H', 'H', 'U', 'H', 'N', 'P', 'H', 'A', 'R', 'u', '_', 'M', 'H', 'c', 'F', 'F', 'F', 'H', 'H', 'H', 'D', 'K', '`', 'B', 'U', 'd', 'G', 'D', 'C', '_', 'U', 'K', '_', '_', '_', '_', '_', 'E', 'c', 'M', '_', 'A', 'A', 'X', 'H', 'E', '_', 'R', 'W', 'L', 'd', '_', 'N', 'H', 'E', '_', 'H', 'H', 'S', 'c', 'c', 'c', 'a', 'A', 'P', 'G', '_', 't', 'F', 't', 'A', 's', 's', 'H', '_', '_', 'S', '_', '_', '_', 'A', '_', 'V', '_', 'C', 'J', 'U', 'A', 'J', 'A', 'e', 'E', '_', 'B', 'u', 'Y', 'A', 't', 'W', 'c', '_', 'U', 'p', 'P', 'E', 'u', 'T', 'c', '_', '`', 'r', 'R', '_', 'P', '_', '_', 'O', 'F', 'E', '_', 'H', 'Q', '_', 'O', 'E', 'R', 'w', 'E', 'Y', 'F', 'H', 'F', 'Z', 'p', '_', '_', 'r', '_', '_', '_', 'a', 'K', 'S', 'E', 'U', '_', 'E', '_', '_', 'H', 'E', 'J', 'T', 'Q', 'E', 'J', 'r', 'W', 'a', 'F', 'J', 'U', 'F', '_', 'L', 'r', 'N', 'r', 'W', 'h', 'r', 'A', 'N', '_', 'A', '_', '_', '_', 'Q', 'r', 'A', 'A', '_', 'M', 'M', 'U', 'F', 'U', 'H', 'H', 'p', '_', '_', 'A', 'd', 'E', 'M', 'A', 'z', 'V', 'Y', '_', 'F', 'N', 'E', '_', 'c', 'c', '_', '_', '_', 'C', 'T', 'O', 'X', 'T', 'F', 't', 'F', 'F', '_', 'W', 'U', 'e', 'H', 's', 'E', 'S', '_', 'b', 'l', 'P', '_', '_', '_', 'R', '_', 'l', '_', 't', 'J', 'L', '_', 'A', 'E', '_', 'A', 't', 'U', '_', 'R', 'i', 's', 'e', 'e', 'e', '_', '_', 'W', 'J', '_', 'U', 's', 't', 'T', '_', 'L', 'X', 's', 's', 'U', 's', 'U', '_', 'U', 'H', '_', '_', 'E', 'L', '_', 'M', 'M', 'J', 'U', 'w', 'h', 'D', 'h', 't', 'H', '_', 'F', 'W', 'r', 'r', 'P', 'R', '_', 'T', 'E', 's', 'U', 'U', 'U', '_', 'P', 'X', 'r', '_', 'V', 'r', '_', '_', 'a', 'A', 'S', 'A', '_', 'J', 'M', 'W', 'L', 't', 'S', 'A', '_', 'A', 'd', 'O', 'Q', 'M', 'G', 'U', 'M', 'W', 'H', 'O', '`', 'X', 's', '_', 'R', 't', 'H', '_', 'B', '_', '_', '_', 'T', 'X', '_', 'q', 'U', 'o', 'M', 'R', 'R', 'M', 'S', 'r', 'r', 'O', 'B', '_', 't', 't', 't', '_', '_', 'r', 'p', 'K', 'Q', 'C', '_', 'J', 'N', 'J', 'E', 'E', 'C', 'I', 'm', 't', 'M', 'i', 'i', 'B', '_', 'E', 'S', 'T', 'R', 'K', 'A', 'M', 'H', 'R', '_', '_', 't', 'd', '_', '_', 'd', 'p', 'O', 'O', 'F', 'J', 'W', '_', '_', '_', '_', 't', 'c', '_', '_', 'a', 'h', 'O', 'E', 'A', 'E', '_', '_', 'W', '_', '_', '_', 'W', 'C', 'W', '_', 'r', 'G', 'Q', '_', 'A', 'T', 's', 'H', 'U', 'O', 'U', 'B', 'U', 'M', 'F', 'W', 'A', 'K', 'E', '_', 'h', 'I', '_', '_', 'L', 'U', 'd', 'R', 'r', 'R', '_', 'r', 't', '_', 'E', 'F', '_', 'U', 'T', 'T', '_', 'O', 'T', 'X', 'R', 'C', 'H', 'U', 'a', "'", 'I', 'A', '_', '_', 'E', 'W', 'F', '_', 'U', 'U', 'J', '_', 'p', 'p', 'Z', '_', '_', 'U', 'c', 'M', 'Z', 'F', 'A', '_', 'W', 'W', 'O', 'n', 'A', '_', 'c', '_', 'W', '_', '_', '_', 'I', 'c', '_', 'p', '_', 'H', 'A', 'U', 't', '_', '_', 't', '_', 'M', 'a', 'H', 'R', '_', '_', 'r', '_', 'E', '_', 'B', 'c', 'A', '_', 'A', 'X', '_', 'p', 'd', '_', 'A', 'p', 'd', 'S', 'L', 'R', 's', 't', '_', 'A', 'N', 'U', 'M', '_', '_', 'U', 'U', '_', 'O', 'A', 'X', 'O', 'K', 'X', 'J', 'E', 'N', '_', 'r', 'a', 'W', '_', '_', '_', 'T', '_', 'a', 'A', 'A', '_', 'U', '_', '_', 'H', 'H', 'U', 'S', 't', '_', 'G', 'A', '_', '_', 'E', 'Q', 'B', 'H', 'B', 'N', '_', 'c', 'H', 'G', 'E', 'U', 'Z', 'O', '_', 'Z', 'A', '_', 'A', 't', 'B', '_', 'A', 'A', 'T', 'r', '_', 'p', 'h', 'K', '_', 'K', 'W', 'X', 'T', 'P', 'E', 'F', '_', 'O', 'Y', 'M', 'P', '_', 'Z', 'R', 'H', 'k', '_', 'E', 'H', '_', 'F', '_', '_', 'U', 'c', '_', 'P', 'G', 'H', '_', '_', 'R', 'Q', 'R', 'M', '_', 'G', 'N', 'U', '_', 'U', 'd', 'J', 'J', 't', '_', 'H', 'H', 'I', '_', 'V', 'M', '_', 'a', '_', 'D', 'A', 'c', '_', 'a', 'A', 'F', 'E', 'E', 'Z', 'C', '_', 't', 'Q', '_', '_', 'B', 'E', 'E', 't', 'E', '_', '_', 'C', 'r', '_', 'B', 'A', '_', 'A', '_', 'l', '_', 'E', 'F', '_', 'F', 'F', 'L', 'C', 's', 't', '_', '_', 'E', 'P', '_', 'W', 'H', '_', 'R', 'R', 'X', 'A', 'J', 'S', 'R', 'N', '_', 'R', 'X', 'T', '_', '_', '_', 'l', '`', 'M', 'F', 'E', 'P', 'A', '_', 'V', 'A', 'M', 'A', 'A', 'Q', 'm', 'H', 's', 'h', '_', 't', 'q', 't', 'K', 'B', 'E', '_', '_', 'Y', 'Z', 'X', 'E', '_', 'c', 'R', 'W', 'J', 't', 'H', 't', 'H', 'O', 'B', '_', 'c', '_', 'T', 'c', 'A', 'A', 'a', '_', 'T', 'O', 'O', 'O', 'O', 't', 'n', 'I', '_', 'H', 'H', 'U', 'r', 'e', 'F', 'r', 'F', '_', 'T', 't', 't', 'T', 'A', '_', 'A', 'E', 'A', 'T', '_', 'K', '_', 'U', 'g', '_', '_', 'F', 'N', '_', 'K', 'E', '_', '_', 'E', 'a', 'F', 'R', 'c', 'r', 'U', 'Y', '_', 't', 'i', 'A', '_', '_', 'G', 't', '_', 'U', 'A', '_', 'S', 'F', 'G', 'K', '_', '_', 'A', 'D', 'C', 'N', 'E', 'T', 'X', 'L', 'H', '_', 'p', 'p', 'p', 'B', 'L', 'H', '_', '_', '_', 'o', 'G', 'h', 's', 'J', 'T', 'a', 'N', 'M', 'K', 'h', '_', 'U', 'l', 'E', 'K', 'Z', 'l', 'K', '`', 'A', 'p', 'U', '_', 'T', '_', '_', '_', 'E', '_', 'W', 'D', "'", 'R', 'O', 'S', '_', 't', 'H', 'E', 'F', 'H', 'n', '_', 'i', '_', 'J', 'Q', 'Q', 'p', '_', '`', 'H', 'W', 'W', 'U', 'V', '_', '_', 'J', 'i', 'B', 'H', 'H', 'H', 'S', 'D', 'K', '_', '_', 'J', '_', 'J', '_', '_', '_', 'U', 'D', 'V', 'H', 'E', 'N', '_', 'U', 'F', 'M', 'R', 's', 'Z', 'M', 'A', 'F', 'M', 'F', 'W', 'c', '_', 't', 'E', 'U', 'H', '_', 'M', '_', 'E', '_', 'C', '_', '_', 'M', '_', '_', '_', 'T', 'A', 'U', 'R', '_', 'A', 'A', 'J', 't', 'U', 'K', 'c', 'R', 'N', 'M', 'B', '_', '_', '_', '_', 'A', 'm', '_', 'B', 'X', 'E', '_', '_', 'Z', '_', 'W', 'O', 'M', 'M', '_', 'B', '_', 'a', 'z', 'R', 'r', 'L', '_', 'D', 'E', '_', 'G', 'H', 'B', 'c', '_', 'L', '_', '_', 'A', 'c', 'J', 'F', 'n', 'W', 'I', 'A', '_', 'i', 'J', 'G', 'T', '_', 'u', 'U', 'U', 'p', '_', '_', 'h', '_', 'U', 'W', 'N', 'W', '_', 'X', 'K', 'M', 'r', 'm', '_', 'S', 'H', 'H', 'r', 'A', '_', 'N', '_', '_', 'r', 'r', 'M', 'U', '_', 'W', 'W', 'A', '_', 'A', 'A', 't', 'A', 'X', 'r', 'A', 'r', 'K', 'U', '_', 'Q', 'J', 'J', 'O', 'K', 't', 's', 'T', '_', 't', '_', 'W', 't', '_', '_', 'D', '_', 'r', '_', 'p', 'p', 'e', 'p', 'H', 'V', '_', 's', 'A', '_', 'r', 't', 'A', '_', 's', 'r', '_', 's', '_', 'q', 'z', '`', 'r', 'S', 'A', 'O', 'J', '_', 'd', '_', 'V', '_', '_', '_', '_', 'a', 'a', 'V', 'Z', '_', 'A', 'V', 'V', 'H', 'U', 't', 'K', 't', '_', '_', 'Y', 'M', 'Z', 'w', 't', 'U', 'X', 'H', 'H', 'E', 's', '_', 'H', 'J', 'H', 'T', 'E', 'F', 't', 'O', 't', 't', '_', 'M', '_', 't', 'X', 'U', 'W', 'N', '`', 'i', '_', '_', '_', '_', 'J', '_', 'H', 'E', '_', '_', 'N', '_', '_', '_', 't', 'A', 'r', 'T', 'Y', 'N', 'r', 'A', 'T', '_', 'S', 'r', 'G', '_', 'Z', 'R', 'F', '_', '_', '`', 'M', 'N', 'M', '_', 'B', 'E', 'D', 'Q', 'E', 'H', 't', '_', 'M', 'c', 'U', '_', 't', 't', 'r', '_', 'F', 'G', '_', 'J', 'U', 'I', 'i', 'X', 'M', '_', 'd', 'A', '_', 'M', 'B', 'F', 'S', 'A', 'X', 'C', 'D', 'X', 'J', 'F', 'R', '_', '_', 'L', 'L', 'W', 'U', 'T', 'W', 'T', 'P', 't', 'E', 'N', 't', 'n', 'H', 'F', 'H', 'r', 'H', 'e', 'A', 'L', 'H', 'X', 't', '_', 'B', 'F', 'w', 'K', 'R', 'U', 'U', 'n', 'E', '_', 'B', 'V', 'A', 'H', '_', 'U', 'C', 'C', 'G', 'F', 'U', 'N', 'J', 'U', 'H', 'P', '_', 'p', 'r', 'p', 'a', '_', '_', '_', '_', 'U', 'H', 'U', '_', 'b', '`', 'F', 'C', 'n', 'w', 'A', '_', 'p', 'T', '_', 'N', 't', 'Q', '_', 'R', 'K', 'A', 'M', 'F', '_', 'l', '_', 'H', '_', 'F', 'G', '_', 'A', 'J', 'N', 'G', '_', '_', '_', '_', 'U', '_', '_', 'V', 'd', 'U', 'H', 'X', 'U', 'N', 'N', '_', 'Q', 'W', '_', 'r', 'H', 'i', 'P', 'i', '_', 'A', 'E', 'A', '_', '_', '_', 'J', '_', '_', 'U', '_', 'q', 'R', '_', '_', 'A', '_', '_', 'A', 'S', 'U', '_', 'H', 'U', '_', 'X', 'A', 'i', 'J', 'e', '_', 'M', 'R', '_', 't', 'a', 'w', 'h', 'n', 'G', 'P', 'n', 'A', '_', 'U', 'L', 'R', 'p', '`', 'J', 'r', 'J', '_', '_', 'J', '_', '_', 'U', 'f', '_', '_', 'C', 'F', 'm', '_', 'A', 'E', 'F', 'B', 'R', 'K', '_', 'B', 'd', 'E', 'J', 'U', '_', 'L', 'U', '_', 'K', 'Z', 'W', 'X', 'W', 'U', 'y', 'r', 'R', 'A', 'U', 'h', 'A', 'a', '_', 'K', 'H', '_', 'A', 'A', 'o', '_', '_', 'E', 'F', 'F', 'B', 'H', '_', 'H', 'h', 'T', 'E', 'Z', 'p', 'A', '_', 'J', 'r', 'e', '_', 'c', 'E', '_', 'J', '_', '_', 'P', 'M', 'J', '_', 'L', 'O', 'O', 'd', 'R', '_', 'R', 'E', 'R', 'B', 'A', 'A', 'W', 'J', '_', '_', '_', '_', 'K', 'D', 'A', '_', 'E', 'G', '_', 'R', '_', 'Z', 'J', 'F', '_', 'U', 's', 'N', '_', 'F', '_', '_', 'd', 'Z', '_', 'T', 't', '_', 'X', 'U', 'P', 'U', 'l', 't', 'T', 'E', 'P', 'H', 'E', 'U', 'E', 'c', 'F', 'D', 'K', 'r', 'a', 'c', '_', 'H', 't', 'F', 'W', 'n', 'W', 'N', '_', 'U', 'b', 'U', 'U', 'B', 'H', 'V', '_', '_', 'K', 'P', 'A', 'U', 'c', '_', 'c', 'U', 'A', 'U', 'F', 'U', 'r', 'c', 'O', 'N', 'e', 'A', 'F', 'E', '_', '_', '_', 'd', 'O', 'S', 'd', 'H', '_', 'r', 'M', 'g', 'M', 'H', 'B', 'W', 'S', 'J', 'J', 'H', '_', 'E', 'F', 'N', 'h', '_', 'W', 'O', 'V', 'U', 'A', 'a', 'a', 'z', '_', 'A', '_', 't', '_', 'D', 'E', 'P', '_', 'w', 'M', 'G', 'D', 'D', 'E', 'P', 'A', '_', '_', 'h', 'c', '_', '_', 't', 'k', 'A', 'E', 'Q', '_', 'A', 'a', 'A', 'Z', 'a', '_', 'F', '_', '_', 't', 'A', 'Y', 'S', '_', '_', 'S', 'W', 'p', 'X', 'O', 'p', 'M', 'A', '_', 'A', '_', 'V', 'U', '_', '`', 'U', 'c', 'c', 'Q', 'r', 'A', 'U', 'T', 'K', 'T', '_', '_', '_', '_', 'c', 'N', 'A', 'A', 'M', 'V', 'F', '_', 'H', 'E', '_', 'Q', 'i', '_', 'r', 'W', '_', 'e', 'a', '_', '_', '_', '_', '_', '_', 'o', '_', 'N', '_', 'R', 'F', 'A', 'E', 'F', 'Z', '_', 'L', 'p', 'P', 'O', 'E', 'D', 'N', 'A', '_', '_', 'C', 'X', 'B', 'B', 'R', 'M', 'Y', 'Y', 'J', '_', 'r', 'r', 'c', 'R', 'V', '_', 'V', '_', 'F', 'F', 'F', 'W', 'F', '_', 'H', 'W', 'M', 't', 'g', '_', 'C', 'F', 'M', 'M', 'W', '_', 'A', 'S', 'J', 'K', 'H', 'a', 'B', 'a', '_', 'a', '_', '_', 's', 'Q', '_', 's', 'O', 'W', 'l', '_', 'X', 'H', 'H', 'r', 'G', 'U', 'V', 'H', 'G', 'H', 'l', '_', '_', 'S', 'U', 'X', 'W', 'u', '_', 'X', 'r', 'A', 'A', 'A', 'T', 'P', 'R', '_', 'E', 'A', 'P', 'h', 'P', 'm', 'A', '_', '_', 'H', 'H', 'A', '_', 'W', 'K', 'G', 'H', '_', 'H', 'U', 'U', 'G', 'Y', 'R', 's', 'L', 'd', 'R', '_', '_', 'd', 'M', 'A', '_', '_', 'U', '_', 'F', 'Q', 'P', 'Z', '_', 't', 'R', 'A', 'W', 'D', '_', 't', 'E', 'P', 'W', '_', '_', '_', '_', '_', 'H', 'd', '_', 'Q', 'A', 'H', 'U', '_', 'Q', 'R', 'B', 'O', 'O', '_', 'n', 'K', '_', 'p', '_', 'A', '_', 'k', 'B', 'c', 'K', 'U', 'R', 'O', '_', 'L', 'O', 'K', 'N', 'H', 'H', 'P', 'O', 'H', 'E', 'c', 'V', 'N', '_', 'U', 'c', 'V', 'U', 'M', 'H', 'Q', 'A', 'C', 's', 'O', '`', 'R', 't', 'W', '_', 'A', 't', 's', 'o', 'r', '_', 's', 'G', '_', 'e', 'X', 'O', 'J', 'r', 'H', '_', 'H', 'e', '_', '_', '_', 'Q', '_', 'H', '_', 'Z', '_', 'W', 'E', 'Y', 'A', 'P', '_', 'A', '_', 'Z', 'd', '_', 'A', 'a', 'a', '_', 'O', 'H', 'W', 'c', 'A', 'H', '_', 'c', 'I', 'S', 'F', 'r', 'F', 'E', '_', 'H', 'A', 's', 'E', '_', 'P', 'U', 'F', '_', '_', 'r', 'M', '_', 'R', '_', 'H', '_', 'r', 'X', 'h', 'P', '_', '_', '_', 'H', '_', 'F', 'p', 'F', '_', 'm', '_', 'i', '_', '_', '_', '_', 'T', 'Z', '_', 'L', 'B', 'E', 'C', 't', 'J', 'N', 't', 'E', 'P', '_', '_', '_', 'B', 'B', 's', '_', 'C', 'W', 'S', 'Y', '_', 'b', 'h', 'H', 'h', '_', '_', 'l', 'H', '_', '_', 'c', 'K', 'c', 'h', 'F', 'T', 'F', 't', 'S', 'S', 'H', 'J', 'E', '_', 'T', 'L', 'X', '_', '_', '_', 'm', 'A', 'M', 'a', 'm', 'K', 'B', 'n', 'C', '_', '_', 'S', 't', 'W', '_', '_', '_', 'D', 'B', 'J', 'F', 'r', '_', 'g', 'J', 'F', 'F', 'O', '_', 'J', 'X', 'O', 'r', '_', 'a', 'L', 'W', 'T', 'G', 'H', 'P', 'V', 'J', 'E', 't', 'T', 'n', '_', 'E', 'J', '_', 'n', '_', 'R', '_', 'U', 's', 'A', '_', 'Q', 'W', 'T', 'c', 'c', 's', 'D', 'E', 'P', 'M', 'I', '_', 'O', 'U', 'E', 'U', 'U', '_', '_', 'N', 'R', 'E', '_', 'c', 'J', 'L', 'K', '_', 'H', '_', 'A', 'A', 'S', 'p', '_', 'p', '_', '_', 'n', 'O', 'R', 'E', 'n', 'E', 'U', '_', '_', 'T', 'R', 'F', 'E', 'p', '_', '_', 'b', '_', 'C', 'N', 'c', 'c', '_', 'p', 'G', 'F', 'r', 'N', 'J', 'U', 'B', 'U', 'X', 'U', 'T', 'P', 'E', 'H', 'W', 'Q', 'P', 'Z', 'T', 'T', '_', 'A', '`', 't', 'r', 't', 'r', 'V', '_', 'R', 'R', 'B', 'Y', 'H', 'Q', 's', '_', '_', 'a', 'c', '_', 'H', '_', 'E', 's', 'W', 'd', 'H', 'U', 'J', 'c', 'J', '_', 'r', 'J', 'e', 'e', 'n', 'U', 'A', 'H', 'Z', 'A', 'A', '_', 'c', 'I', 'K', '_', 'D', '_', 'p', '_', 'J', 'F', 'P', 'M', '_', 'U', '_', 'a', '_', '_', '_', 'O', 'e', '_', '_', 'J', '_', 'F', 'K', '_', 'F', '_', 'E', 'E', 'm', 'P', 'd', '_', '_', '_', 'c', '_', 'T', '_', '_', '_', 'V', 'r', '_', 'A', '_', 'Z', 'n', 'U', 'U', 'W', '_', 'A', 'G', 'r', 'p', 'l', 'o', '_', '_', 'F', '_', 'P', 'I', 'A', 'I', 'Q', '_', 'D', 'M', '_', '_', 'V', '_', '_', 'Q', '_', 'J', 'F', 'U', 'd', 'H', 'M', 'd', 'O', '_', '_', 'F', 'M', 'L', 'e', '_', '_', '_', 'r', 'F', 'J', 'R', '_', '_', 'S', 'V', 'U', '_', 'h', 'F', 't', 't', '_', '_', 'W', 'O', 'w', 'M', 'N', 'V', '_', 'E', 'E', 'H', 'h', '_', 'h', '_', '_', '_', 'E', '_', 'V', 'T', 'M', 'E', '_', 'r', '_', 'p', 'h', 'T', 'e', '_', 'H', 'H', 'J', 'c', 'L', 'M', '_', 'B', 'E', 'E', 'p', '_', '_', 'p', 'K', '_', '_', '_', 'h', '_', 't', 'U', 'E', 'R', 'O', 'T', '_', 'R', 'c', 'A', '_', 'J', 'R', 'W', 'r', '_', 'R', 'i', '_', '_', 'F', '_', '_', '_', '_', 'k', '_', 'l', 'T', '_', 'J', 'R', 'J', 'B', 'H', 'H', 'M', 'h', 'F', 'z', '_', 'Z', 't', 'Q', 'r', 'A', 'M', '_', 'H', 'V', '_', 'v', 'M', 'D', 'X', 'W', 'Q', 'V', 'Q', 'c', 'M', 'R', 'V', 'Q', 'F', 'F', 'E', 'F', 'o', 'H', 'M', 't', '_', 'U', '_', 'S', 'L', '_', 'M', 'w', 'w', 'J', 'w', 'i', '_', '_', 'i', '_', 'G', 'r', '`', '_', 'L', 'r', 'C', 'H', 'a', 'E', 't', 'I', 'A', 'A', 'A', 'A', '_', 'U', '_', 'K', 'K', 'c', '_', 'r', 'd', 'H', 's', '_', 'c', 'E', 'E', '_', 'E', 'N', 'M', 'F', 'A', 'H', 'W', 'A', 'N', 'X', 'A', 'T', '_', 'T', 'J', 'p', 'H', '_', 'M', '_', 'M', '_', 'm', 'A', 'K', '_', 'J', 'M', 'O', '_', 'U', 'H', 'E', 'V', '_', 'A', 'V', 'J', 'A', 'A', 'R', 'N', 'H', 'G', '_', 'H', 'T', 'h', 'Z', 'H', 'H', 'Q', 'H', 'h', '_', 'J', '_', '_', 'W', 'W', 'M', 'B', '`', '`', '_', '_', 'S', '_', 'r', 'R', '_', 'Z', '_', '_', 'O', 'P', 'n', 'H', 'J', '_', 'U', '_', 'O', 'r', 'U', 'c', 'r', 'N', '_', 'M', 'W', 'n', '_', 'e', 'N', 'e', '_', '_', 'D', '_', 'd', 'd', 'K', 'Y', 'G', 'O', 'W', 'H', 'P', 'U', 'U', 'l', 'c', 'K', 'E', 'U', 'T', 'A', 'r', 'H', '_', 'H', '_', '_', '_', '_', 'T', 'T', 'M', 'T', 'r', 'p', 'A', 'P', 'U', '_', 'n', 'F', '_', '_', 'P', 'E', 'A', '_', '_', 'Q', 'Q', 'J', '_', '_', 'W', 'A', 'U', 'E', '_', 'M', 'A', '_', 'Z', 'a', 'G', '_', 'K', 'p', '_', 'a', '_', 'a', '_', '_', '_', '_', '_', 'X', 'O', 'M', 'X', 'c', 'E', 'R', 'M', 'H', 'Q', '_', 'L', '_', '_', 'W', '_', 'T', 'S', '`', 'r', '_', '_', '_', 'L', 'G', 's', 'Q', 'A', 'c', '_', 'D', 'c', 'V', '_', 'r', '_', 'M', 'a', 'F', 'B', '_', 'T', 'Z', 'T', 'E', 'A', '_', 'a', 't', 'l', 'C', 'U', 'X', 't', 'N', 'N', 'M', 't', 'F', 'T', 'A', 'R', '_', 'N', '_', 'M', '_', 'H', 'p', '_', 'p', '_', 'W', 'W', 'H', 'F', 'F', 'r', '_', 'H', 'E', '_', 'R', 'F', 'Q', 'F', 'F', 'P', '_', '_', 't', '_', 'S', 's', 'U', 'h', 'h', 't', 'U', 'R', '_', '_', 'Y', 'Q', 'H', 'H', 'A', 'K', 'E', 'A', 'U', 'C', '_', 'h', 'H', 'c', 'D', 'E', '_', 'U', 'X', 'W', '_', 'M', 'X', 'H', 'A', 'M', 'N', 'N', 'T', '_', '_', 'P', 'A', 'A', 'Z', 'C', 'H', '_', 'H', '_', '_', 'B', '_', 'T', 'l', '_', 'E', 'r', 'A', '_', 'F', 'a', 'A', 'w', 'w', 'K', 'N', 'e', 'C', '_', 'R', 'J', 'Z', 'A', 'H', 'F', '_', 'D', 'a', '_', 'D', 't', 'M', 'R', 'D', 'E', 'D', '_', 'b', 'c', '_', 'r', '_', 'W', 'M', 'W', 'C', 'O', '_', '_', 't', 'U', 'U', 'J', 'J', 'J', '_', '_', 'K', 'E', 'n', '_', 'M', 'A', 'W', 'L', 'H', 'O', 'U', 'l', 'Q', 'X', '_', 'A', 'A', '`', 'A', '_', 'W', 'H', 'U', 's', 'E', 'T', 'U', 'U', '_', 'U', '_', 'r', 'C', 'r', 'Q', 'V', '_', 'B', 'w', 'R', 'Q', '_', '_', 'm', '_', '_', 'A', '_', 'R', 'R', 'V', 'S', '_', '_', 'U', 'M', '_', '_', 'U', '_', 'E', '_', '_', 'A', '_', 't', 'H', 't', 'U', 'U', 'p', 'P', 'Q', '_', '_', 'K', '_', 'E', 'R', 'F', 'W', "'", 'r', 's', 'Z', 'J', 't', '_', 'X', '`', 'H', 'W', 'W', 'A', 'H', 'H', 's', 'W', '_', 'O', 'J', 's', '_', 'C', 'W', '_', 'U', 'r', 'T', 'K', 'Q', 'G', 'H', '_', '_', 'L', 'G', 'd', '_', '_', 'd', 'U', '_', '_', 'p', 'V', 'p', 'p', 'p', 'F', 'p', '_', 'E', '_', 'X', 'V', 'N', 'E', 'E', 'r', 'E', 'A', 'U', '_', 'N', '_', 'a', '_', 'e', 'J', '_', '_', 'u', '_', '_', 'H', 'H', 'H', 'E', 'T', 'c', '_', 'U', 'Y', 'W', 'L', '_', 'H', 'h', 'H', 'H', '_', 't', 't', 't', 'U', 'h', 'U', '_', 'F', 'Q', 'W', 'P', 'r', 'r', 'a', 'm', 'U', 'Z', 't', 'h', '_', 'B', '_', 'A', '_', '_', 'A', 'A', 'H', 'A', 'L', '_', 'R', 'F', '_', '_', 'F', '_', '_', 'W', 'k', 'A', '_', 'X', '_', '_', 'U', 'R', 'U', 'Z', '_', 'U', 'Z', 'S', '_', '_', '_', 'L', 'F', '_', 'P', 'M', 'W', 'E', '_', 'Q', 'S', 'K', 'H', '_', 'C', '_', '_', 'R', 'H', 'N', 'Q', 'f', 'E', 'K', 't', 't', 'H', 'h', 't', '_', '_', 'S', 'J', 'O', 'A', 'c', 'Y', '_', 'n', 'i', 'Q', '_', '_', 'h', 't', '_', '_', 'O', 'E', '_', '_', 'K', 'M', 'c', 'A', '_', 'T', 'h', 'W', 'J', '_', 'R', 's', 'T', 'R', '_', 't', 'A', 'r', 'R', 'Q', '_', '_', 't', 'P', 'E', 'U', '_', 'J', 'K', 'O', 'G', 'X', 'P', 'W', 'O', 'R', '_', 'A', '_', 'B', 'P', 'r', 'i', 'Z', 'J', 'H', 'k', 'K', 'r', 'T', 'c', 't', 'J', '_', '_', 'T', 'Q', 'S', 'M', 'O', 'O', 'K', 'S', 'A', 'W', 'Y', 'L', 'c', 'P', 'Z', '_', 'T', 'r', 'Q', 'r', '_', '_', '_', '_', 'O', '_', 'r', '_', 'O', 'A', 'a', 'A', 'a', '_', '_', '_', 'E', 'H', '_', 'A', 'l', 'H', 'N', 'B', 'G', 'J', 'p', 'A', 'V', 'p', 'X', 'e', 'e', 'E', 'E', 'e', 'A', '_', 'H', 'H', '_', 'T', 'K', 'W', 'H', '_', '_', 'U', 'E', 'c', 'T', '_', 'M', 'Q', 'U', 'A', 'E', 'A', 'X', 'E', '_', '_', 'A', 'B', 'a', 'l', 'U', 'r', 'r', '_', '_', '_', 'Z', 'V', 'Q', 'A', '_', '_', '_', '_', 't', '_', '_', 't', 't', '_', '_', 'U', '_', 's', 'F', '_', 'E', 'l', '_', '_', 'R', '_', 'L', '_', '_', '_', 'A', 'c', 'A', 'S', 'E', 'o', 'T', 'n', 'J', 'A', 'K', 'Q', '_', 'Z', '_', 'Q', 'a', 'c', 'A', 'p', 'a', 'p', '_', 'Q', 'H', 't', 'F', 'E', 'a', 'S', 'H', 'J', 'n', 'c', 'T', 'F', 'F', 'E', 'W', '_', 'O', '_', 'F', '_', 'H', 'a', 't', 'a', 'c', 'X', '_', 'c', 'M', 'U', '_', '_', '_', 'C', '_', 'c', 'T', '_', 'R', '_', 'p', 'h', 'H', 'T', 'O', 't', 's', '`', 'N', 'B', 'Z', '_', '_', 'E', 'o', 'F', 'H', 'O', 'U', 'M', 'T', 'E', 'A', 'm', 'M', 'H', 'm', 'P', 'c', 'm', 'O', 'M', '_', 'H', '_', 'F', '_', '_', '_', '_', 'K', 'd', 'E', '_', 'c', '_', 'R', 'W', '_', 'F', 'F', 'p', 'H', 'p', 'S', '_', 'T', 'r', '`', '_', '_', '`', '_', 'K', 'M', 'a', 't', 'A', 'O', 'A', 'H', '_', 'o', 's', '_', 'F', 'F', 'F', 'H', '_', '_', '_', 'H', 'F', 'r', 'V', 'H', '_', 'T', '_', 'W', '_', 'h', 's', '_', '_', '_', '_', '_', 'J', 'B', 'T', '_', 'L', 'A', 'A', 'Z', '_', 'H', 't', '_', 'l', 'B', 'r', 'r', 'h', '_', 'A', '_', 'h', 't', 'W', 'A', '_', 't', 't', 'A', 'T', 'B', '_', '_', 's', 'A', 'h', 'h', 'K', 'r', 'r', 'E', 'W', 'H', 'N', 'M', 'r', 'A', 'M', 'U', 'I', 'E', 'H', '_', '_', 'V', 'K', 'Q', 'E', 'L', 'H', 'H', 'H', 'E', 'D', 'H', 'K', 'U', 'A', 'O', 'L', 'O', '_', '_', '_', 'e', 'n', '_', 'W', 'e', 'H', '_', 'i', 'F', 'u', 'l', 'W', '_', 'E', 'i', '_', '_', 's', 'A', 'p', 'r', 'S', 'm', 'A', '_', 'M', 'Q', '_', 'W', 'P', '_', 'D', 'K', 't', 'y', '_', 'Y', 'H', 'Q', '_', 'A', 'H', 'r', 'I', '_', 'K', 'E', '_', '_', 'T', '_', '_', 'Q', 'r', 'r', 'r', 'O', '_', 'R', 'E', 'F', 'K', 'H', '_', 'U', '_', 'U', 'e', 'V', 'A', 'L', '_', 'T', '_', 'C', '_', 'B', '_', '_', 'C', 'n', 'c', 'a', '_', 'p', 'q', '_', 'J', 'H', '_', 'O', '_', 't', 'J', 'Q', 'p', '_', '_', '_', '_', 'h', 'H', 'p', 'O', 'W', 'C', 'J', 'f', 'F', 'R', 's', 'I', 'D', '_', '_', '_', '_', 'X', 'd', 'R', 'U', 'R', 'F', 'H', 'R', 'P', 'S', 'R', '_', '_', 'W', 'G', 'J', 't', 'E', 't', 'N', 'r', 'R', '`', '_', 'R', 'K', 'd', '_', '_', 'C', 'W', 'R', 'r', 'r', 'U', '`', 'i', 'W', 'K', '_', 'L', 'W', '_', '_', '_', 'U', 'A', '_', '_', '_', 'K', 'R', 'R', '_', 'E', 'E', 'M', 'Q', 'M', 'A', 'R', 'J', 'J', 'Z', 'E', 'Z', '_', 'E', '_', '_', 'E', '_', 'R', 'B', 'p', 'W', 'r', 'r', 'd', '_', 'S', 'F', 'p', 'J', 'H', 'k', '_', 'c', '_', '_', '_', '_', 'F', 'O', '_', '_', '_', 'R', 'H', 'U', 'R', 'A', 'A', 'H', 'J', 'A', 'G', 'A', '_', 'A', 'A', '_', 'A', '_', '`', 'N', 't', '_', 'h', 'A', '_', 'A', 'K', 'c', '_', 'A', 'I', '_', 'c', '_', '_', 'Q', 'l', 'G', '_', '_', 'R', 'Q', 'U', 'G', 'R', '_', 'F', 'E', '_', '_', 'F', 'U', 'U', 'H', 'A', 'N', '_', 'W', 'E', '`', 'y', 'R', '_', 'M', '_', '_', 'l', 'M', '_', 'R', 'M', 'Q', 'F', 'O', 'C', 'Z', 'K', 'Z', '_', 'R', '_', '_', 'X', 'o', 'G', 'Q', 'L', 'J', 'J', 'J', 'M', 'm', '_', 'Y', 'c', '_', '_', 'U', 'c', '_', 'c', 'b', '_', 'W', 'A', 'A', 'b', 'Q', 'P', 'M', '_', 'G', 'T', 'B', 'T', '_', 'd', 'P', 'T', '_', 'Q', 't', 'A', 'A', 'U', 'Q', 'H', 'U', '_', 's', '_', 'A', '_', 'K', '_', 'Y', 'r', 'A', 'O', 'T', 'A', 'h', 'r', 'P', 's', 't', 'V', 'B', 'S', 'p', 'H', '_', 'M', 'A', 'C', 'H', 'C', 'c', 'A', 'c', 'R', 'P', 'Q', 'W', '_', '_', 's', '_', 'U', 'E', 'd', 'H', 'O', 'Y', 'E', 'T', '_', 'O', 'F', 'M', 'O', '_', 'V', 'M', 'Q', '_', 'p', '_', '_', '_', 'c', '_', '_', 'c', 'd', 't', 't', 'V', 'F', '_', '_', 'E', 'A', '_', 'H', '_', 'H', 'B', 'D', 'B', 'N', 'd', 'X', '_', 'M', 'd', 'E', 'A', 'P', 'G', 'P', 'Z', 'O', 'P', 'A', 'd', 'A', '_', 'M', 'F', 'J', 'p', '_', 'p', 'R', 'n', 'D', 'F', 'O', 's', '_', 'O', 'M', '_', 'P', 'd', 'A', 'S', 'd', 'T', '_', 'D', '_', 'E', 'B', 'l', '_', 'A', '_', 'F', 'R', 'H', '_', 't', 'h', 'e', 'U', 'h', '_', '_', 'G', '`', 'T', 'Q', '_', 'A', 't', 'r', '_', 'A', 'P', '_', 'U', '_', 'C', 's', '_', 'R', '_', 'e', '_', 'X', 'H', '_', 'A', '_', 'U', 's', 'S', 'N', 'C', 'H', '_', 'V', 'L', 'P', '_', '_', 'E', '_', 'N', 'B', '_', 'p', '_', '_', 'h', '_', '_', 'O', '_', '_', 'F', 'T', 'J', 'E', '_', 'M', 'n', 'F', '_', 'X', '_', 'O', 'L', 'n', 'B', 'P', 'm', 'M', 'C', 'A', 'T', '_', 'Y', '_', '_', 'D', 'U', 'Q', '_', 'N', '_', 'O', 'A', 't', '_', 'V', 'X', '_', '_', 'Z', 'B', 'V', '_', '_', 't', 't', 'e', 'U', 'c', '_', 'E', 'R', '_', 'U', 'X', 'O', '_', 'U', '_', 'B', 'F', 'T', 'A', 'F', 'T', '_', 'R', 'P', 'N', 'm', 'M', '_', '_', 'T', 'T', 'K', 'Q', 'E', 'r', 'A', 'A', 'r', 'K', 'H', 'A', 'C', 'c', 'G', 'G', 'X', 'n', 'M', 's', '_', 'c', '_', '`', 'S', 'U', '_', 'r', '_', 'U', 'V', 'Q', 'U', 'r', '_', 'p', 'r', 'c', '_', 'K', 'L', 'F', 'F', 't', 'A', 'V', 'K', 'p', 'T', 'B', 'H', '_', 'N', '_', 'e', '_', '`', 'c', 'c', 'c', 'i', 'F', '_', 'U', 'H', 'V', 'E', 'G', 'a', 'a', 'a', 'M', 'a', '_', 'J', 'Q', 'A', 'I', 'A', 'H', '_', 'D', 'A', '_', 'P', 'n', 'M', 'A', 'V', 'W', 'M', '_', 'I', 'K', 'n', 'K', 'K', 'n', 'n', '_', 'V', 'A', 'A', 'A', 'F', 'H', 'B', 'B', 'A', 'C', 'p', '_', 'Q', 'F', 't', 'U', 'H', 'J', 'R', 'W', 'W', 'F', 'F', 'w', 'T', '_', '_', 'E', '_', '_', 'F', 'H', 'F', 'C', '_', 'K', 'J', 's', 'A', 'R', 't', 'Q', 'A', 'N', 'L', 'J', '_', 'R', 'P', 'U', '_', 'R', 'W', '_', '_', '_', 'c', 't', 'H', 'h', 'p', 'p', '_', '_', '_', '_', 't', 'c', 'O', 'O', 'Q', 'O', 'c', '_', 'i', 'W', 'D', '_', 'Q', 'c', 'L', 'R', '_', '_', '_', '_', 'U', '_', 'w', '_', 'R', 'W', 'L', 'A', 'G', 'r', '_', 'a', 'V', 'F', 'E', 'A', 'E', 'K', 'V', 'h', 'J', 'd', 'r', '_', '_', 'F', '_', 'r', '_', 'p', 'U', 'M', 'U', 'U', 'X', 'R', 'd', '_', 'B', 'H', 'A', 'J', 'R', 'M', 'M', 'P', 'H', 'c', 'c', 'C', 't', 'U', 'U', 'K', 'U', 'U', 'G', 'A', 'V', 'A', '_', '_', 'F', 'A', 'A', 'n', 'K', 'O', '_', 'r', '_', 'J', 'F', 'W', 'S', '_', 'w', 'i', 'a', 'K', 'M', 'A', 'p', 'p', 'U', 'A', 'J', '_', 'K', '_', 'w', 'w', 'O', 'r', 'n', 't', 'e', 'K', '_', 'A', 'L', 'P', 'c', 'U', 'C', 'C', '_', '_', 'W', 's', '_', 'e', '_', 'd', '_', '_', 'A', 'A', 'c', 'k', 'W', '_', '_', '_', 'A', '_', '_', 'E', 'H', 'B', 'E', '_', 'Z', 'F', '_', 'J', '_', 'S', 'h', 'i', 'e', 'Z', 'F', 'P', 'W', 't', 'A', 't', 'E', '_', 'p', '_', 'W', '_', 'J', 'Z', 'p', 'p', 'L', '_', 'L', 'O', 'U', 'c', '_', 'E', 'A', '_', 'c', 'W', 'e', 'E', 'e', 'H', '_', 'h', 'r', '_', 'A', 'H', 'k', 'h', 'h', 'h', 'H', 'M', 'A', 'S', 'M', 'r', 'R', '_', '_', 'p', 'X', '_', '_', 'R', 'J', '_', '_', 'B', 'M', 'F', 'F', 'm', '_', '_', 'M', '_', 'K', '_', 'U', 'U', 'M', 'p', 't', '_', '_', '_', '_', 'B', 'P', 'J', 'w', 'w', 'O', 'h', 'H', 'U', 'A', 'M', 'A', 'H', 'd', 'F', 'P', 'T', 'E', 'Y', 'U', 'N', '_', 'i', 'p', '_', 'G', '_', '_', 'E', '_', '_', 'M', 't', 'A', '_', 's', 'X', 'Y', 'E', 'B', 'M', 'U', '_', 'w', '_', 'r', 'W', 'C', 'H', 'M', 'p', '_', '_', 'B', 'W', '_', '_', 'N', '_', 't', 'p', 'p', '_', 'H', 'A', 'D', 'A', 't', '_', 'L', 'r', 'i', 'J', 'd', 'T', '_', 'X', 'M', 'R', 'L', 'J', 'n', 'a', 'a', 'S', 't', '_', 'r', '_', '_', 'A', '_', '_', 'r', 'a', 'K', '_', '_', 'd', '_', '_', 'W', 'Z', 'p', 'A', 't', 'A', '_', '_', 'r', 'H', 'H', 'F', '_', 'T', '_', 'H', 'A', 'R', 'U', 'Q', '_', 'E', 's', 'R', 'F', 'W', 'H', '_', 's', 't', 'V', 'A', '_', 'A', 'V', 'U', 'F', '_', '_', '_', '_', 'i', 'J', 't', 'a', 'N', 'a', 'n', 'E', '_', 'V', '_', '_', '_', 'E', 't', 't', 'B', 't', '_', 'E', 'W', '_', '_', 'M', '`', 'K', 'H', '_', '_', 'O', 'n', '_', 'K', 'Y', 'r', '_', 'r', '_', '_', 'U', '_', 'A', '_', '_', '_', 'K', 'h', 'E', 'H', 'c', '_', 'B', 'K', 'U', '_', '_', 'm', 'H', '_', 'L', 'c', 'F', 'c', 'O', '_', 'a', 'A', 'c', 'H', 'X', 'A', 'h', 'U', '_', 'H', 'L', 'E', 'T', 'O', '_', 'S', 'U', '_', 'U', 'L', 'U', 'N', 'p', '_', 'p', '_', '_', 'p', 'r', 'K', 'E', 'K', 'T', 'P', '_', 'J', 'V', 'a', 'E', 'B', 'Q', '_', 'l', 'G', '_', 'r', 'E', '_', 'E', 'p', 'U', 'E', 'J', 'B', 'd', 'T', '_', 'Y', 'L', 'U', 'd', '_', 'E', 't', '_', 'R', 'N', 'E', 'R', 'U', 'T', 'R', 'u', 'Y', 'U', '_', 'J', 'h', 'H', 'A', 'T', '_', 'n', 'c', 'H', 'E', 'N', 'r', 'H', 'U', '_', '_', 'Y', 'F', 'V', '_', '_', '_', 'A', 'E', 'F', 'P', '_', '_', 'A', 'U', '_', 'G', '_', '_', '_', 'X', 'W', '_', 'Z', 'W', 'h', 'H', 'H', 'K', '_', 'Z', 'r', '_', 'H', 'P', 'M', 'A', 'Z', 'F', 'T', 'E', '_', '_', 'B', '_', 'U', 'K', 'H', '_', '_', '_', 'B', 'N', 'E', '_', 'p', 'p', 'J', 'E', 'H', '_', '_', 'R', 'P', 'r', 'p', '_', 'U', 'a', '_', 'A', '_', '_', 'H', 'K', 'a', 'U', '_', 'd', 'R', '_', 'p', 'p', 'l', '_', 'H', 'E', '`', '_', '_', 'K', 'c', '_', '_', '_', 'X', 'S', 'A', 'H', 't', '_', 't', 't', 'O', 't', '_', 'Q', 'M', 'G', 'n', '_', 'K', '_', 'U', 'U', 'W', 'Q', 'Q', 'U', 'q', 'c', 'Z', 's', '_', '_', 'R', '_', 'h', 'H', '_', 'H', 'B', 'Q', 'H', 'E', '_', 'W', 'Y', 'T', 'W', '_', 'A', '_', 'Q', 'H', '_', 'A', 'H', '_', 'R', '_', '_', 'H', 'F', 'R', 'J', 'G', 'J', 'U', 'a', 'K', 'A', 'A', 'V', 'F', 'U', 'F', 'c', 's', 'G', 'S', '_', '`', '_', 'e', '_', 't', 'F', 't', 'B', '_', 'B', 'r', 't', 'd', 'X', 't', '_', 'N', '_', 'E', 'e', 'H', '_', 'm', 'X', '_', 'P', 'c', 'Q', 'E', '_', 'H', 'A', 'h', '_', '_', '_', '_', 'U', 'A', '_', 'E', '_', '_', 'r', 's', '_', '_', 'E', '_', 'b', 'b', 'H', 'U', '_', 'Q', 'p', 'Y', '_', 'p', 'O', '_', '_', 'r', 'H', 'T', 'F', 'X', 't', '_', 'm', 'B', 't', 'R', '_', 'T', 'p', 'A', '_', 'J', 'd', '_', 'T', 'G', '_', 'G', 'Z', 's', 'G', '_', 'h', 't', 'Z', 't', 'Q', 'H', 'A', 'n', '_', '_', 'U', 'Z', 'c', 'T', 'R', 'M', 'h', 'c', 'Z', 'U', '_', 'F', 'F', 'F', 'J', 'T', 'r', 'R', 'e', 'L', 'E', 'i', 'X', '`', 'N', '_', 'N', '_', 'W', 'l', 'a', '_', 's', 'B', '_', '_', '_', 'c', 'c', 'e', 'Q', 'a', '_', 't', 'E', 'T', 'r', '_', 'Q', 'P', '_', 'Q', '_', '_', 'U', 'Z', 'D', 'R', 'R', 'H', 'K', '_', 'U', 'r', '_', '_', '_', 'W', 'D', 'P', 'c', 'c', 'B', '_', 't', 'G', 'B', 'F', 'C', '_', 'O', '_', 't', '_', 'i', 'R', '_', 'R', 'A', 'J', '_', 'B', '_', '_', '_', '_', 'G', 'A', 'Z', '_', 'V', 'F', 'K', '_', 'F', '_', 'C', '_', 'W', 'F', 't', 'H', 'H', 'A', '_', 'Q', 'R', 's', 'P', 'B', 'p', 'H', 'n', 'i', 'd', 'b', '_', '_', 'N', 'E', 'A', 'J', 'B', 'O', 'I', 's', 't', 'O', '_', 'R', 'Q', '`', 'E', '_', 'F', 'F', 'P', 'A', 'P', 'P', '_', 'B', 'H', 'H', 'A', 'W', 'M', 'W', '_', '_', 'L', '_', 'O', 'A', '_', '_', 'E', '_', 'H', 'h', 'J', '_', 'H', '_', 'H', 'n', 'O', 'n', 'C', 'C', 'W', 'C', 'W', 'N', '_', '_', '_', 'h', 'G', '_', '_', 'A', '_', 'F', '`', 'H', 'R', 's', '_', 'J', 'A', 'A', 'C', 'F', '_', 'F', 'O', 'H', 'H', 'S', 'H', 'H', '_', '_', 'h', 'H', 'M', 'E', 'P', 'M', 'H', 'A', 'a', 'A', 'A', 'r', 'Y', 't', 'A', 'A', 'c', 't', 'J', 'W', 'd', 'G', 'I', 'W', 'r', 's', 's', '_', 'M', 'N', 'N', '_', 'Z', '_', '_', '_', 'B', 'r', 'C', 'A', 'n', '_', 'A', 'n', '_', 'D', 'c', '_', 'A', 'J', 'A', '_', 'A', 'H', 'H', 'm', 'W', 'm', 'r', 'W', 'U', '_', 'H', 'U', '_', '_', 'e', 'F', 'J', 'H', 'Q', '_', '_', 'W', 'e', '_', 'H', '_', 'U', 'm', 'U', 'e', 'C', '_', 'U', 'A', 'J', 'L', 'H', '_', 'U', '_', 'A', 'a', 'T', '_', '_', 'A', 'r', 'R', 'E', 'H', 'T', 'N', 'V', 'Q', 'C', 'T', '_', '_', 't', 't', 'K', '_', '_', 'c', 'c', 'U', 'U', 'R', 'M', 'A', 'V', 'z', 'A', 'A', '_', '_', 'A', 'p', 'P', 'L', 's', '_', 'R', 'R', 'D', 'V', 'M', 'W', 'W', 'c', 'A', '_', 'E', 'P', '_', 't', 'c', 'H', 'h', '_', '_', 'M', 'Q', 'r', 'E', 't', 't', 'V', '_', 'J', 'd', 'E', 'r', 'J', 'N', '_', 'w', 'A', 'p', 'm', '_', '_', 'L', 'i', 'A', 'M', 'l', '_', '_', '_', 'd', 'h', 'K', 'H', '_', '_', '_', 'm', 'C', 'p', 'T', 'm', 'H', '_', 'A', '_', '_', 't', 's', '_', 'T', '_', '_', 'F', 'G', 'J', '_', 'd', '_', '_', 'J', 'P', 'N', '_', '_', 'K', '_', 't', 't', 'A', '_', 'R', '_', 'R', 'G', 'A', '_', 'F', 'l', 'M', 'U', '_', '_', 'V', '_', 'Q', 'i', 'O', '_', 'G', 'K', 'G', 'V', 'F', 'E', 'W', 'U', 'A', 'A', 'F', 'n', '_', 't', '_', 'r', 'p', '_', '_', '_', '_', 'o', 'O', 'E', 'X', 'M', 'L', 'E', 'c', 'D', 'F', 'p', '_', '_', 'H', 'X', 'e', 'R', '_', 'A', 'T', 't', 't', 'R', 'R', 't', 'U', '_', 'H', 'K', 'Y', '_', 'F', 'F', '_', '_', 'H', '_', '_', 'W', 'n', 'm', 'Q', 't', 'O', 'b', '_', '_', 'a', 'E', 'A', '_', '_', '_', 'r', 'R', 'r', 'u', 'p', 'A', 'p', 'H', 'E', 'T', 'V', 'J', '_', '_', 's', 'U', 'M', 'M', 'B', '_', 't', 'E', 'r', '_', '_', 'C', 'p', '_', 'O', 'Z', 'W', 'A', '_', 'M', '_', 'Z', 'n', '_', 'B', 'A', '_', 'h', 'R', '_', 'R', 'L', 'F', 'F', 'K', 'H', 'T', 'A', 'T', 'B', '_', 'U', 'p', '_', '_', '_', 'N', 'F', 'P', 'N', 'H', 'W', '_', 'R', '_', '_', 'R', '_', 'F', '_', 'E', '_', 'E', '_', '_', 'M', '_', '_', '_', 'E', 'N', '_', 'H', 't', '_', '_', 'p', '`', '_', 'J', 'G', 'h', 'm', 'W', 'W', 'm', 'E', '_', 'm', 'M', '_', 'U', 'B', 'K', 'U', 'r', 'D', 'J', '_', 'K', '_', '_', 'a', 'A', 'M', 'W', 'A', 'T', 'P', 'U', 't', 'N', 'A', 'e', 't', 'm', '_', '_', '_', 'P', 'F', 'i', 'u', 'P', 'd', 'U', 'E', 'G', 'F', '_', '_', 'R', 'R', 'W', 'C', 'K', '_', '_', 'P', 'U', 'c', '_', 'D', 'A', 'S', 'A', 't', 'M', 'X', 'U', 'Z', '_', 'F', 'V', 'F', 'p', '_', 'H', '_', '_', 'U', 'h', 'c', 'h', '_', 'F', '_', 'w', 'w', 'd', 'w', '_', '_', 'a', 'U', 'A', 'A', 'F', 'E', 'p', '`', 'K', 't', 'A', 'D', 't', 'P', 'A', 'X', 'M', '_', '_', 'M', 'c', 'H', 'Z', '_', '_', '_', 'T', '_', 'A', 'K', '_', '_', 'u', 'P', '_', 'Q', 'H', 'H', '_', '_', 'Z', 'O', '_', '_', 'H', 'U', '_', 'A', 'F', '_', 'i', 't', 'E', 'r', '_', '_', '_', 'r', '_', 'O', 'C', '_', 'V', 'O', 'R', 'U', 'U', '_', 'a', 'W', 'W', 'W', 'W', 'L', 'V', 'D', 'K', 'H', '`', 'M', 'R', 'O', 'Z', '_', '_', 'A', 'C', '_', 'T', 'M', 'C', 'M', 'P', '_', 'M', 'u', 'a', 't', '_', '_', '_', 'n', '_', 'n', 'E', 'b', 'W', 'N', 's', '_', 'W', 'M', 'U', 'H', 'X', 'G', 'Q', '_', '_', 'p', '_', 'p', 'K', 'U', '_', '_', 'R', 'H', 'U', '_', '_', 'F', '_', 'h', 'F', 'F', '_', 's', '_', '_', 'i', 'O', '_', '_', '_', 'U', 's', 'A', '_', 'A', 'Z', '_', '_', 'H', 'H', 'J', 'F', 'F', '_', 'F', 'A', '_', 'n', 'A', 'a', 'L', 'U', 'C', 'L', 'F', 'Q', 'c', 'U', '_', '_', '_', 'e', '_', '_', 's', 's', 'H', 'r', 'I', 't', '_', 'A', 'B', 'X', '_', 'N', 'K', 'V', '_', 'F', '_', 'R', 'H', 'Q', 'A', 'O', 'S', 'A', '_', 'A', 'm', '`', 'R', '_', 'H', 'c', 'V', 'H', 'M', 'N', 'Q', 'r', '_', '_', 'N', '_', 'm', '_', 'T', 'V', '_', 'A', '_', 'T', '_', '_', 'F', 'F', 'F', 'F', 'R', 'F', 'D', 'e', 'n', 'M', 'r', 'A', 'D', 'T', 'B', 'Y', 'D', 'Y', 'X', 'U', 'V', '_', '_', '_', 'J', 't', '_', '_', 'E', '_', 't', 't', '_', '_', 'T', 'c', '_', '`', "'", 'E', 'S', 'D', 'H', 'U', 'r', 'W', 'K', '_', 'Z', 'K', 'O', 'P', 'A', 'c', 'F', '_', 'A', 'A', 'E', '_', 'E', 'W', '_', '_', '_', '_', 'h', '_', 'h', 'H', 'A', 'G', 'A', 'd', 'U', '_', 'U', '_', 'J', 'A', 'E', '_', 'r', 'F', 'Q', 'F', 'r', 'D', 'F', 'i', 'H', 'H', 'n', 'B', '_', '_', 'M', 'S', 't', '_', 'a', 'A', 'E', '_', 'd', 'E', 'E', '_', '_', '_', 'r', '_', 'U', '_', 'E', 'H', 'F', 'H', 'H', 'W', 'H', '_', '_', 'B', 'K', 'J', 'K', 'E', 'F', 'F', 'M', '_', 'Q', 'D', 'h', '_', '_', '_', '_', 'c', '_', '_', 'U', '_', '_', 'L', '_', '_', 'U', '_', 'L', 'J', 'O', 'E', '_', '_', '_', 'A', '_', '_', 'd', 's', '_', 't', '_', 'r', 'E', 'O', 'M', 'h', 'E', 'T', 'F', 'F', 'H', 'F', 'H', 'L', 'H', 'W', 'V', 'Z', 'F', 'L', 'F', 'F', 'M', 'E', 'F', 'J', 'A', 'K', 'F', 'E', 'E', 'H', 'A', 'H', 'a', 'r', 'c', 'h', 'H', '_', 'E', 'N', 'A', 'O', 'A', 'A', '_', '_', 'O', 'c', 'C', 'O', '_', 'W', 'p', '_', 'e', '_', 'F', '_', 'T', 'D', 'O', 'M', 'V', 'M', 'r', 'H', 'F', 'r', 'U', 'G', 'R', 'U', 'H', 'U', 'e', 'H', '_', 'e', '_', 'J', 't', 'X', 'b', 'E', 'J', '_', 'N', 'K', '_', 'r', 'c', 'y', 'N', 'K', 'R', 'T', 'W', 'U', 'Q', 'c', 'c', 'c', 'C', '_', 'P', 'D', '_', '_', 'A', 'e', 'N', 'e', 's', 'X', '_', 't', '_', '_', '_', '_', 'M', 'E', 'F', 'Z', 'W', 'K', 'K', 'C', 'E', '_', 'r', 'P', '_', 'W', '_', 'V', '_', 'Y', '_', 'J', 'B', '_', 'S', 's', 't', 'F', 'X', 'S', 'S', 'F', 'l', '_', 'F', 'A', '_', 'H', 'Q', 'H', 'Q', 'J', 't', '_', 'Q', '_', 't', 'E', 'Z', '_', 'U', '_', 'a', 'H', 'U', '_', 'H', 'd', 'R', 'P', 'R', 'r', '_', '_', '_', '_', 'A', 'P', 'L', 'H', '_', 'r', 'l', '_', 'n', 'F', 'A', '_', '_', '_', 'F', 'R', '_', 'A', 'S', 'O', '_', 's', '_', 'a', '_', '_', '_', '_', 'm', 'e', 'J', 'H', 'F', '_', 'y', '_', 'H', 'O', 'p', 'G', '_', 'V', 'n', 'u', '_', 'D', 'L', 'K', 'k', 'B', '_', 'd', '_', '_', 'K', '_', '_', '_', 'u', '_', 'A', '_', '_', '_', 'U', '_', 'p', '_', '_', 'H', 'K', 'M', 'H', 'X', 'H', '_', 'K', 'c', '_', 'l', 'Z', 'J', 'U', 'Q', '_', 'V', 'L', '_', '_', 'B', 'A', 'H', 'Z', 'Q', '_', 'T', 'S', 'M', '_', '_', '_', 'N', 'W', '_', '_', 'M', 'm', '_', '_', 'F', 'E', 'B', 'Q', 'E', 'U', 's', '_', 'Y', '_', 'A', 'C', 'Z', 'r', 't', 't', '`', 'o', '_', '_', '_', 'l', 'J', '_', 'R', 't', 'L', 'E', 'N', 'Q', 'D', '_', 'Z', 'R', 'P', 'E', 'p', 'Q', 't', '_', 'c', 'H', 'A', 'K', '_', '_', 'N', 'U', 'A', '_', 'J', 'W', '_', 'S', '_', 'A', 'E', 'K', 'A', 't', 'A', 't', 't', 'U', '_', '_', '_', 'A', 'A', 'K', 'Q', '_', '_', 'K', '_', '_', '_', 'r', '_', 'H', 'F', 'F', 'M', 'H', 'Z', 'E', 'r', 'Y', 'E', '_', 'X', 'H', '_', 'R', '_', 'Y', '_', 'K', 'a', 'E', 'E', 'N', 'O', 'O', 'H', 'W', 'p', 'p', '_', 'V', 'L', 'K', 'c', '_', 'O', 'h', 'P', 'I', 'H', '_', 'I', 'H', '_', '_', 'H', '_', 'Q', 'U', 't', 'A', '_', 'w', 'T', 'C', 'E', 'T', 'M', '_', 'm', 't', 't', '_', 'm', '_', '_', 'A', 'G', 'T', 'G', 'd', '_', 'E', 'h', 'O', '_', 't', 'O', '_', 'A', 'r', 'V', '_', 'r', 'A', 'w', 'M', 'a', '_', 'c', 'U', 'u', 'P', 'U', 'V', 'H', 'd', '_', 'U', 'N', 'c', 'H', 'R', 'r', '_', 'a', '_', 'c', 'U', 'p', 'd', 'p', 'p', 'H', 'F', '_', 'd', '_', 'W', 'K', 'c', 'K', 'K', '_', 'M', '_', '_', '_', 'X', 'p', '`', '_', '_', 't', 'H', 'T', 'A', 't', '_', '_', 'D', 'F', '_', 'h', 'r', '_', "'", 'Z', '_', 'R', 't', 'J', 'E', '_', 'a', '_', '_', 'M', 'n', 'c', 'e', 'O', 'c', 'Z', 'H', 'S', 'H', 'n', 'R', 'd', '_', '_', '_', 't', 'K', 'r', 't', 'p', 'P', 'P', '_', 'G', 'A', 'a', 'E', 'U', 'I', 'Y', '_', 'Y', '_', '_', '_', '_', 't', 'G', 'E', '_', 'M', 'Q', '_', 'K', '_', 'P', 'O', '_', 'r', 't', 'U', 'd', 'U', 'd', 'M', 'O', '_', 'U', '_', 'M', 'A', 'L', 't', 'r', 'r', 'J', 'U', 'H', 'G', '_', 'p', 'U', 'M', 'H', 'F', 'F', 't', '_', 't', 't', 'F', 'l', 'W', 'O', '_', 'L', '_', 'p', 'U', 'F', 'W', 'n', 'D', 'U', 'V', 'F', '_', 'H', 'd', '_', 'J', 'L', 'A', 'S', 'G', 'X', 'o', 'X', 'X', 'X', 'O', 't', 'F', 'A', 'm', 'O', 'd', 'B', 'R', '_', '_', '_', 'X', 'E', 'X', 'J', 'A', 'N', '_', '_', 'p', '_', 'K', '_', '_', '_', '_', 'l', 'W', '_', '_', 'Q', 'a', 'I', 'A', '_', "'", '_', 'a', '_', 'J', 'p', 'U', 't', 'A', 'O', 'J', '_', 's', '_', 'J', '_', 'A', 'Y', 'Z', 'D', 'c', 'R', 'J', 'K', 'T', '_', '_', 'E', 'Z', 'J', 'B', 't', 'R', 'Y', 't', 'r', 'a', "'", '_', '_', 'a', '_', '_', 'A', 'P', 'O', 'H', '_', 'Z', 'M', 'm', 'U', 'A', '_', 'U', '_', 'J', '_', 'U', 'X', '_', 'U', '_', 'V', 'a', 'R', 'P', 'V', 'H', 'p', 'p', '_', 'b', 'U', '_', 'A', 'A', '_', 'M', 'f', 'c', 'H', '_', '_', 'M', '_', '_', 'E', '_', 'r', '_', '_', 'U', 'i', '_', 'T', 'p', 'E', '_', 'd', '_', '_', '_', 'D', 's', 'U', 'R', '_', 't', 'h', 'r', 't', 'U', '_', 'M', 'M', 'R', '_', 'H', 'N', 'p', '_', '_', 't', '_', 'M', 'U', 'm', 'M', 'm', 'M', 'U', 'F', 'R', '_', 'r', '_', 'r', 'O', '_', 'K', 'Z', 't', 'P', 'D', 'r', 'g', '_', '_', 'K', 'g', 'a', 'U', 'p', 'S', 'V', '_', '_', '_', 'A', 'T', 'M', 'T', '_', '_', 'E', '_', 'Z', 'E', 'H', 'H', 'c', '_', '_', 'Q', 'F', 'A', 'P', 'r', 'c', 'I', '_', 'L', '_', '_', '_', 'J', 'w', 'W', 'J', '_', 'c', "'", 'c', 'c', '_', '_', 'U', 'U', '_', 'd', 'p', 'E', 'r', 'U', 'F', 'A', 'E', 'E', 'B', 'O', 'E', 'P', '_', 'T', 't', '_', 'R', 't', 'J', 'N', 'Z', '_', 'K', 'E', 'H', '_', 'n', 'Q', 'E', 'W', 'R', '_', 'e', 'G', 'F', 'N', 'o', 'N', 'H', 'O', '_', '_', '_', '_', '_', 'P', 'E', 'R', 'H', 'T', 'E', 'A', 'E', 'a', 'a', 'T', 'A', 'K', 'A', 'A', 'M', 'a', 'R', 'H', 'U', 'H', '_', 'B', 'A', 'A', '_', '_', 'A', '_', '_', '_', 'S', '_', 'H', 'F', '_', 'M', '_', 'H', 'D', '_', 'X', 'Q', 'A', 'R', 'T', 'H', 'B', '_', '_', 'V', 'K', 'N', 'A', '_', 'a', 'D', 'T', 'B', 'k', '_', 'H', '_', 'D', 'T', 'r', 'H', 'F', 'F', 'W', 'A', 'E', 'F', 'K', 'H', 'r', 'r', 'X', '_', 'A', 'X', 'K', 'Q', 'h', 'n', 't', 't', 'U', 'E', '_', 'U', 'm', 'e', '_', 'W', 'J', 'H', 'R', '_', 'R', 'R', '_', 'R', 'E', 'K', '_', '_', '_', 'H', 'r', '_', 'V', 'Q', 'h', '_', '_', 'h']


#Test

In [3]:
len(output_letter)

10510

In [4]:
7875/10510

0.7492863939105614

In [ ]:
input_ids = tokenizer(input_texts[i], return_tensors="pt").to(model.device)

outputs = model.generate(
    **input_ids,
    max_new_tokens=1)
print(tokenizer.decode(outputs[0])[-20:])

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


 Correct response: _


In [ ]:
prompt = """You are playing a game of Hangman.

Your task is to guess a single character.

The word has a certain number of letters.
The current state of the word is shown with guessed letters filled in and blanks for the unknown letters.
The number of incorrect guesses remaining is listed.
All letters that have been guessed so far are listed.

You will format your response as a single uppercase letter at the end

The word has 6 letters.
The current state is: P _ _ _ O _
Incorrect guesses remaining: 5
Guessed letters: [E, P, O]

Correct response:"""

text = tokenizer.apply_chat_template(
    prompt,
    tokenize=False,
    # add_generation_prompt=True,
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=1
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

del model_inputs, generated_ids
gc.collect()

print(generated_text)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Sure


In [ ]:
input_text = """You are playing a game of Hangman.

Your task is to guess a single character.

The word has a certain number of letters.
The current state of the word is shown with guessed letters filled in and blanks for the unknown letters.
The number of incorrect guesses remaining is listed.
All letters that have been guessed so far are listed.

You will format your response as a single uppercase letter at the end

Here are a few examples to guide you:

Example 1
Prompt:
The word has 5 letters.
The current state is: A _ _ _ E
Incorrect guesses remaining: 4
Guessed letters: [A, E]
Correct response: R

Example 2
Prompt:
The word has 8 letters.
The current state is: B A _ _ _ _ A
Incorrect guesses remaining: 3
Guessed letters: [B, A]
Correct response: N

Example 3
Prompt:
The word has 7 letters.
The current state is: C H A _ _ _ _
Incorrect guesses remaining: 2
Guessed letters: [C, H, A]
Correct response: L

Example 4
Prompt:
The word has 6 letters.
The current state is: _ O O _ L _
Incorrect guesses remaining: 1
Guessed letters: [O, L]
Correct response: G

Example 5

The word has 10 letters.
The current state is: _ _ _ A _ _ _ I _ N
Incorrect guesses remaining: 4
Guessed letters: [A, I, N]
Correct response: S

Now, let's play the game.

The word has 6 letters.
The current state is: P _ _ _ O _
Incorrect guesses remaining: 5
Guessed letters: [E, P, O]

Correct response:"""
input_ids = tokenizer(input_text, return_tensors="pt").to(model.device)

outputs = model.generate(
    **input_ids,
    max_new_tokens=1)
print(tokenizer.decode(outputs[0])[-20:])


Correct response: R


In [ ]:
# prepare the model input
prompt = """You are playing a game of Hangman.

The word has 6 letters.
The current state is: _ _ _ _ _ _
Incorrect guesses remaining: 6
Guessed letters: [ ]

Your task is to guess one letter. Output a single letter at the end."""
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)


thinking content: <think>
Okay, let's see. I need to figure out the correct letter to guess in this Hangman game. The word has 6 letters, and the current state is that there are four underscores, and the remaining guesses are 6. The guessed letters are empty. So the goal is to determine which letter to guess next.

Wait, but the problem says "output a single letter at the end". Hmm, maybe I need to pick a letter that's not guessed yet, not in the wrong position, and not part of the wrong letters. Let me think.

The initial state shows that there are four underscores. So the word has 6 letters, and the underscores are the letters that haven't been guessed yet. So the possible letters to guess could be any of the 6 letters that haven't been guessed yet, but also not in the wrong positions. Wait, but the problem says that the current state is "_ _ _ _ _ _", so maybe the underscores are the letters that need to be filled in. The remaining guesses are 6. So the user has 6 guesses left, but 

In [ ]:
# prepare the model input
prompt = """You are playing a game of Hangman.

The word has 6 letters.
The current state is: P _ _ _ O _
Incorrect guesses remaining: 5
Guessed letters: [E, P, O]

Your task is to either guess one character or guess the full word.
Format your response as follows:
- To guess a character, output a single letter preceded by "CHARACTER:"  at the end (e.g., "A").
- To guess the full word, output the word preceded by "WORD:" at the end (e.g., "WORD: PYTHON").

Your guess:"""
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)


thinking content: 
content: <think>
Okay, let's see. I need to figure out the answer to this Hangman game. The user has provided the current state, the number of incorrect guesses left, and the letters that have been guessed. The word is 6 letters long, and the current state is P _ _ _ O _. So far, the letters guessed are E, P, O. The remaining letters in the word are probably A, N, T, H, and maybe another one? Wait, the current state has P, then underscores, then O. So the letters in the word are P, A, N, T, H, and O? Wait, no, the original word is 6 letters. Let me think again. The current state is P _ _ _ O _. So the letters guessed are P, O. The current state shows that the word has P, then three underscores, then O. So the letters not guessed yet are A, N, T. Because the word is 6 letters. So the word is P A N T H O? Or maybe P A N T O H? Wait, the current state shows O at the end. So the word is 6 letters. Let me count: P, then three underscores, then O. So the word is P followed

In [ ]:
model_inputs

{'input_ids': tensor([[151644,    872,    198,   2610,    525,   5619,    264,   1809,    315,
          40775,   1515,    382,    785,   3409,    702,    220,     21,  11931,
            624,    785,   1482,   1584,    374,     25,    393,    716,    716,
            716,    506,  22983,  40468,  60537,   9664,     25,    220,     20,
            198,  16780,  21712,  11931,     25,    508,     36,     11,    393,
             11,    506,   2533,   7771,   3383,    374,    311,   2987,   7942,
            825,   3668,    476,   7942,    279,   2480,   3409,    624,   4061,
            697,   2033,    438,  11017,    510,     12,   2014,   7942,    264,
           3668,     11,   2550,    264,   3175,   6524,  52480,    553,    330,
          15237,  37397,   2974,    220,    518,    279,    835,    320,     68,
           1302,   2572,    330,     32,  38609,     12,   2014,   7942,    279,
           2480,   3409,     11,   2550,    279,   3409,  52480,    553,    330,
           722

https://github.com/huggingface/trl